In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#
# ==================================
# SETUP: LIBRARIES AND HYPERPARAMETERS
# ==================================
#
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Hyperparameters ---
# These are the key settings for our GAN.
# We can tune these to improve results.

# Training parameters
epochs = 25
lr = 0.0002 # Learning rate, a key parameter for GAN stability
batch_size = 128

# Model parameters
image_size = 64  # We'll resize MNIST images to this size
image_channels = 1 # MNIST is grayscale, so 1 channel
latent_dim = 100 # Dimension of the random noise vector (z)

# --- 2. Device Setup ---
# We want to use the GPU if it's available, as it's much faster.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
#
# ==================================
# STEP 1: DATA PREPARATION
# ==================================
#
# We define a series of transformations to apply to the images.
transform = transforms.Compose([
    transforms.Resize(image_size),      # Resize to our desired image size
    transforms.ToTensor(),              # Convert image to a PyTorch Tensor (values 0-1)
    transforms.Normalize(
        [0.5], [0.5]                    # Normalize to [-1, 1] range
    )
])

# Download the MNIST dataset
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Create a DataLoader to handle batching
dataloader = DataLoader(mnist_dataset, batch_size=batch_size, shuffle=True)

# Let's visualize a batch of real images to see what we're working with
real_batch = next(iter(dataloader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("A Batch of Real MNIST Images")
# We need to un-normalize to display them correctly
grid = make_grid(real_batch[0][:64] * 0.5 + 0.5, padding=2, normalize=True)
plt.imshow(np.transpose(grid.cpu(), (1, 2, 0)))
plt.show()

In [ ]:
#
# ==================================
# STEP 2: THE GENERATOR
# ==================================
#
# The Generator's job is to create realistic images from random noise.
# It uses ConvTranspose2d layers to "upsample" from a latent vector to a full image.

class Generator(nn.Module):
    def __init__(self, latent_dim, channels, img_size):
        super(Generator, self).__init__()
        self.img_size = img_size
        self.channels = channels

        # We build the network layer by layer.
        # It's a series of blocks that progressively increase the image size.
        self.model = nn.Sequential(
            # Input is Z, going into a convolution
            # Block 1: latent_dim -> 512 x 4x4
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            # Block 2: 512 x 4x4 -> 256 x 8x8
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            # Block 3: 256 x 8x8 -> 128 x 16x16
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            # Block 4: 128 x 16x16 -> 64 x 32x32
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            # Block 5: 64 x 32x32 -> 1 x 64x64
            nn.ConvTranspose2d(64, channels, 4, 2, 1, bias=False),
            nn.Tanh() # Tanh activation to scale output to [-1, 1]
        )

    def forward(self, z):
        # z is the input noise vector
        img = self.model(z)
        return img

In [ ]:
#
# ==================================
# STEP 3: THE DISCRIMINATOR
# ==================================
#
# The Discriminator's job is to classify images as real or fake.
# It's a standard CNN.

class Discriminator(nn.Module):
    def __init__(self, channels, img_size):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            # Input: 1 x 64x64
            nn.Conv2d(channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # State: 64 x 32x32
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # State: 128 x 16x16
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            # State: 256 x 8x8
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            # State: 512 x 4x4
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid() # Sigmoid to output a probability (0=Fake, 1=Real)
        )

    def forward(self, img):
        # The input is an image
        validity = self.model(img)
        # We flatten the output to a single value per image in the batch
        return validity.view(-1)

In [ ]:
# #
# # ==================================
# # STEP 4: TRAINING THE GAN
# # ==================================
# #

# # --- 1. Weight Initialization ---
# # As per the DCGAN paper, it's good practice to initialize weights from a
# # Normal distribution with mean=0, stdev=0.02.
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         nn.init.normal_(m.weight.data, 0.0, 0.02)
#     elif classname.find('BatchNorm') != -1:
#         nn.init.normal_(m.weight.data, 1.0, 0.02)
#         nn.init.constant_(m.bias.data, 0)

# # --- 2. Initialize Models & Components ---
# generator = Generator(latent_dim, image_channels, image_size).to(device)
# discriminator = Discriminator(image_channels, image_size).to(device)

# generator.apply(weights_init)
# discriminator.apply(weights_init)

# # Loss function
# criterion = nn.BCELoss() # Binary Cross-Entropy Loss

# # Optimizers (Adam is a good choice for GANs)
# # We need separate optimizers for G and D
# optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
# optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# # A fixed noise vector to see the generator's progress over time
# fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

# # --- 3. The Training Loop ---
# print("Starting Training Loop...")
# for epoch in range(epochs):
#     for i, (real_images, _) in enumerate(dataloader):

#         # ---------------------
#         #  TRAIN DISCRIMINATOR
#         # ---------------------

#         # Send real images to the device
#         real_images = real_images.to(device)

#         # Create labels for real and fake images
#         # Real images get label 1, fake images get label 0
#         real_labels = torch.ones(real_images.size(0), device=device)
#         fake_labels = torch.zeros(real_images.size(0), device=device)

#         # --- Train with real images ---
#         optimizer_D.zero_grad()

#         # Pass real images through the discriminator
#         d_output_real = discriminator(real_images)
#         # Calculate loss on real images
#         errD_real = criterion(d_output_real, real_labels)
#         errD_real.backward()

#         # --- Train with fake images ---
#         # Generate a batch of fake images
#         noise = torch.randn(real_images.size(0), latent_dim, 1, 1, device=device)
#         fake_images = generator(noise)

#         # Pass fake images through the discriminator
#         # We use .detach() on fake_images because we don't want to calculate
#         # gradients for the generator at this stage.
#         d_output_fake = discriminator(fake_images.detach())
#         # Calculate loss on fake images
#         errD_fake = criterion(d_output_fake, fake_labels)
#         errD_fake.backward()

#         # Total discriminator loss is the sum of real and fake losses
#         errD = errD_real + errD_fake
#         optimizer_D.step()

#         # -----------------
#         #  TRAIN GENERATOR
#         # -----------------
#         optimizer_G.zero_grad()

#         # We need to run the fake images through the discriminator again
#         d_output_on_fake = discriminator(fake_images)

#         # **The Generator's Goal**: To fool the discriminator.
#         # It wants the discriminator to output 1 (real) for its fake images.
#         # So, we calculate the generator's loss using REAL labels (1s) for the fake images.
#         errG = criterion(d_output_on_fake, real_labels)

#         # Calculate gradients for the generator and update its weights
#         errG.backward()
#         optimizer_G.step()

#         # --- 4. Logging and Visualization ---
#         if i % 100 == 0:
#             print(
#                 f'[{epoch}/{epochs}][{i}/{len(dataloader)}] '
#                 f'Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}'
#             )

#     # At the end of each epoch, generate images with the fixed noise
#     # so we can see how the generator is improving.
#     with torch.no_grad():
#         fake_samples = generator(fixed_noise).detach().cpu()

#     # Create a grid of images
#     img_grid = make_grid(fake_samples, padding=2, normalize=True)

#     # Save the grid to a file
#     save_image(img_grid, f"mnist_fake_epoch_{epoch}.png")

#     # Display the grid (optional, but great for notebooks)
#     plt.figure(figsize=(8,8))
#     plt.axis("off")
#     plt.title(f"Generated Images at Epoch {epoch}")
#     plt.imshow(np.transpose(img_grid, (1, 2, 0)))
#     plt.show()

# print("Training finished!")
#
# ==================================
# STEP 4: TRAINING THE GAN
# ==================================
#

# --- 1. Weight Initialization ---
# As per the DCGAN paper, it's good practice to initialize weights from a
# Normal distribution with mean=0, stdev=0.02.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# --- 2. Initialize Models & Components ---
generator = Generator(latent_dim, image_channels, image_size).to(device)
discriminator = Discriminator(image_channels, image_size).to(device)

generator.apply(weights_init)
discriminator.apply(weights_init)

# Loss function
criterion = nn.BCELoss() # Binary Cross-Entropy Loss

# Optimizers (Adam is a good choice for GANs)
# We need separate optimizers for G and D
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# A fixed noise vector to see the generator's progress over time
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

# --- 3. The Training Loop ---
print("Starting Training Loop...")
for epoch in range(epochs):
    for i, (real_images, _) in enumerate(dataloader):

        # ---------------------
        #  TRAIN DISCRIMINATOR
        # ---------------------

        # Send real images to the device
        real_images = real_images.to(device)

        # Create labels for real and fake images
        # Real images get label 1, fake images get label 0
        real_labels = torch.ones(real_images.size(0), device=device)
        fake_labels = torch.zeros(real_images.size(0), device=device)

        # --- Train with real images ---
        optimizer_D.zero_grad()

        # Pass real images through the discriminator
        d_output_real = discriminator(real_images)
        # Calculate loss on real images
        errD_real = criterion(d_output_real, real_labels)
        errD_real.backward()

        # --- Train with fake images ---
        # Generate a batch of fake images
        noise = torch.randn(real_images.size(0), latent_dim, 1, 1, device=device)
        fake_images = generator(noise)

        # Pass fake images through the discriminator
        # We use .detach() on fake_images because we don't want to calculate
        # gradients for the generator at this stage.
        d_output_fake = discriminator(fake_images.detach())
        # Calculate loss on fake images
        errD_fake = criterion(d_output_fake, fake_labels)
        errD_fake.backward()

        # Total discriminator loss is the sum of real and fake losses
        errD = errD_real + errD_fake
        optimizer_D.step()

        # -----------------
        #  TRAIN GENERATOR
        # -----------------
        optimizer_G.zero_grad()

        # We need to run the fake images through the discriminator again
        d_output_on_fake = discriminator(fake_images)

        # **The Generator's Goal**: To fool the discriminator.
        # It wants the discriminator to output 1 (real) for its fake images.
        # So, we calculate the generator's loss using REAL labels (1s) for the fake images.
        errG = criterion(d_output_on_fake, real_labels)

        # Calculate gradients for the generator and update its weights
        errG.backward()
        optimizer_G.step()

        # --- 4. Logging and Visualization ---
        if i % 100 == 0:
            print(
                f'[{epoch}/{epochs}][{i}/{len(dataloader)}] '
                f'Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}'
            )

    # At the end of each epoch, generate images with the fixed noise
    # so we can see how the generator is improving.
    with torch.no_grad():
        fake_samples = generator(fixed_noise).detach().cpu()

    # Create a grid of images
    img_grid = make_grid(fake_samples, padding=2, normalize=True)

    # Save the grid to a file
    save_image(img_grid, f"mnist_fake_epoch_{epoch}.png")

    # Save the models
    torch.save(generator.state_dict(), f"generator_epoch_{epoch}.pth")
    torch.save(discriminator.state_dict(), f"discriminator_epoch_{epoch}.pth")

    # Display the grid (optional, but great for notebooks)
    plt.figure(figsize=(8,8))
    plt.axis("off")
    plt.title(f"Generated Images at Epoch {epoch}")
    plt.imshow(np.transpose(img_grid, (1, 2, 0)))
    plt.show()

print("Training finished!")


In [ ]:
#
# ==================================
# STEP 5: IMAGE GENERATION
# ==================================
#
# Put the generator in evaluation mode
generator.eval()

# Generate a batch of new images
with torch.no_grad():
    # Create random noise
    new_noise = torch.randn(64, latent_dim, 1, 1, device=device)
    # Generate images
    generated_images = generator(new_noise).detach().cpu()

# Un-normalize the images to the [0, 1] range for visualization
generated_images = generated_images * 0.5 + 0.5 

# Display the final generated images
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Final Generated Images")
plt.imshow(np.transpose(make_grid(generated_images, padding=2, normalize=True), (1, 2, 0)))
plt.show()

In [ ]:
#
# ==================================
# STEP 6: EXPORT AND RELOAD
# ==================================
#

# --- 1. Export the Generator's State ---
# This saves all the learned weights and buffers of the model.
generator_path = "/kaggle/working/generator_epoch_25.pth"
torch.save(generator.state_dict(), generator_path)
print(f"Generator model state saved to {generator_path}")


# --- 2. Reload the Generator for Inference ---
# In a real application, you would do this in a separate script.
#
# IMPORTANT: You first need to have the model's class definition.
# So you must have the 'Generator' class code available.

# a. Create a new instance of the Generator model
reloaded_generator = Generator(latent_dim, image_channels, image_size).to(device)

# b. Load the saved state dictionary
reloaded_generator.load_state_dict(torch.load(generator_path))

# c. Set the model to evaluation mode
reloaded_generator.eval()

print("Generator reloaded successfully!")

# --- 3. Test the Reloaded Generator ---
# Let's prove it works by generating images again.
with torch.no_grad():
    test_noise = torch.randn(16, latent_dim, 1, 1, device=device)
    reloaded_images = reloaded_generator(test_noise).detach().cpu()
    reloaded_images = reloaded_images * 0.5 + 0.5

plt.figure(figsize=(4, 4))
plt.axis("off")
plt.title("Images from Reloaded Generator")
plt.imshow(np.transpose(make_grid(reloaded_images, padding=2, normalize=True), (1, 2, 0)))
plt.show()

In [ ]:
# ==================================
# STEP 6: EXPORT AND RELOAD
# ==================================

# --- 1. Export the Generator's State ---
# Cette section sauvegarde les poids appris du modèle générateur.

# Spécifie le chemin où le modèle sera sauvegardé
generator_path = "/kaggle/working/generator_epoch_25.pth"  # Alternative : utiliser pathlib pour plus de portabilité -> Path("generator_epoch_48.pth")

# Sauvegarde le dictionnaire d'état (poids et buffers) du générateur
torch.save(generator.state_dict(), generator_path)  # Alternative : torch.jit.save(torch.jit.script(generator), path) pour sauvegarder le modèle complet (structure + poids)

# Affiche une confirmation que le modèle a bien été sauvegardé
print(f"Generator model state saved to {generator_path}")  # Alternative : logging.info(...) pour une gestion plus propre des logs

# --- 2. Reload the Generator for Inference ---
# Cette section montre comment recharger un générateur sauvegardé,
# typiquement dans un autre script, pour générer des images.

# IMPORTANT : la classe Generator doit être définie dans le script courant,
# car on a besoin de connaître l'architecture exacte du modèle.

# a. Crée une nouvelle instance du générateur (même architecture que l’original)
reloaded_generator = Generator(latent_dim, image_channels, image_size).to(device)  
# Alternative : sérialiser l’architecture avec torch.save(generator, path), puis torch.load(path), mais cela a moins de contrôle et peut poser problème si le code change.

# b. Charge les poids sauvegardés dans cette nouvelle instance
reloaded_generator.load_state_dict(torch.load(generator_path))  
# Alternative : ajouter strict=False si les architectures diffèrent légèrement (mais attention à la cohérence des résultats)

# c. Passe le modèle en mode évaluation (important pour désactiver le dropout, etc.)
reloaded_generator.eval()  # Alternative : si tu veux générer avec dropout (ex: test-time augmentation), ne pas appeler eval()

# Confirmation que le modèle a été rechargé avec succès
print("Generator reloaded successfully!")  # Alternative : logging pour journaliser l'étape

# --- 3. Test the Reloaded Generator ---
# On vérifie que le modèle rechargé fonctionne bien en générant de nouvelles images.

# Désactive la grad pour ne pas calculer de gradients (plus rapide et mémoire réduite)
with torch.no_grad():  # Alternative : torch.inference_mode() (plus restrictif et encore plus optimisé pour l'inférence)

    # Génère un batch de bruit aléatoire
    test_noise = torch.randn(16, latent_dim, 1, 1, device=device)  
    # Alternative : torch.normal(mean, std, size=(...), device=device) pour plus de contrôle sur la distribution du bruit

    # Génère des images à partir du générateur rechargé
    reloaded_images = reloaded_generator(test_noise).detach().cpu()  
    # Alternative : sans detach() si tu veux faire un backward ensuite (peu probable ici)

    # Remet les pixels dans la plage [0, 1] (si normalisé en [-1, 1] avant)
    reloaded_images = reloaded_images * 0.5 + 0.5  # Alternative : reloaded_images.add(1).div(2) (même effet mais syntaxe différente)

# Affiche les images générées sous forme de grille
plt.figure(figsize=(4, 4))  # Alternative : modifier figsize pour avoir une grille plus grande ou plus petite
plt.axis("off")  # Alternative : plt.axis("on") si tu veux voir les axes
plt.title("Images from Reloaded Generator")  # Alternative : ajouter des infos dynamiques dans le titre (ex: date, epoch)

# Affiche la grille dans le bon format pour matplotlib
plt.imshow(np.transpose(make_grid(reloaded_images, padding=2, normalize=True), (1, 2, 0)))  
# Alternative : torchvision.utils.save_image(...) pour enregistrer la grille sur disque au lieu de l'afficher

plt.show()  # Alternative : plt.savefig("output.png") pour sauvegarder directement l’image plutôt que de l’afficher


In [ ]:
# ==================================
# STEP 6: EXPORT AND RELOAD
# ==================================

# --- 1. Export the Generator's State ---
# Cette section sauvegarde les poids appris du modèle générateur.

# Spécifie le chemin où le modèle sera sauvegardé
generator_path = "/kaggle/working/generator_epoch_48.pth"  
# Alternatives : 
# - Utiliser `pathlib.Path("generator_epoch_48.pth")` pour une meilleure compatibilité multiplateforme
# - Ajouter un timestamp dynamique au nom du fichier avec `datetime.now().strftime(...)` pour versionner automatiquement

# Sauvegarde le dictionnaire d'état (poids et buffers) du générateur
torch.save(generator.state_dict(), generator_path)  
# Alternatives :
# - torch.save({'model': generator.state_dict(), 'epoch': current_epoch}, path) pour sauvegarder aussi des métadonnées (utile pour reprise d'entraînement)
# - torch.jit.save(torch.jit.script(generator), path) pour exporter un modèle complet prêt pour la production (nécessite que le modèle soit scriptable)
# - torch.save(generator.state_dict(), open(generator_path, 'wb')) pour plus de contrôle sur la méthode d’ouverture du fichier

# Affiche une confirmation que le modèle a bien été sauvegardé
print(f"Generator model state saved to {generator_path}")  
# Alternatives :
# - logging.info(...) pour une gestion centralisée et configurable des logs
# - Ajouter une vérification que le fichier existe via `os.path.exists(...)` pour valider la sauvegarde

# --- 2. Reload the Generator for Inference ---
# Cette section montre comment recharger un générateur sauvegardé,
# typiquement dans un autre script, pour générer des images.

# IMPORTANT : la classe Generator doit être définie dans le script courant,
# car on a besoin de connaître l'architecture exacte du modèle.

# a. Crée une nouvelle instance du générateur (même architecture que l’original)
reloaded_generator = Generator(latent_dim, image_channels, image_size).to(device)  
# Alternatives :
# - Passer des hyperparamètres via un fichier de config (YAML, JSON) pour plus de flexibilité
# - torch.load() si le modèle complet a été sérialisé (structure + poids), bien que cela soit moins recommandé

# b. Charge les poids sauvegardés dans cette nouvelle instance
reloaded_generator.load_state_dict(torch.load(generator_path))  
# Alternatives :
# - strict=False si vous avez modifié légèrement l'architecture (attention : cela peut provoquer un comportement inattendu)
# - torch.load(..., map_location=torch.device('cpu')) pour recharger sur CPU même si le modèle a été sauvegardé depuis le GPU

# c. Passe le modèle en mode évaluation (important pour désactiver le dropout, etc.)
reloaded_generator.eval()  
# Alternatives :
# - Ne pas appeler `eval()` si vous voulez conserver des comportements stochastiques comme le Dropout (utile en test-time augmentation)
# - Utiliser `model.train(False)` (équivalent à `.eval()`)

# Confirmation que le modèle a été rechargé avec succès
print("Generator reloaded successfully!")  
# Alternatives :
# - logging.info("Generator reloaded...") pour un suivi plus rigoureux
# - Afficher également la taille ou le résumé du modèle via `print(reloaded_generator)` ou `summary(...)`

# --- 3. Test the Reloaded Generator ---
# On vérifie que le modèle rechargé fonctionne bien en générant de nouvelles images.

# Désactive la grad pour ne pas calculer de gradients (plus rapide et mémoire réduite)
with torch.no_grad():  
# Alternatives :
# - torch.inference_mode() pour une version encore plus optimisée (depuis PyTorch 1.9)
# - Pas de contexte du tout si vous voulez analyser les gradients pour autre chose (ex : étude du comportement du modèle)

    # Génère un batch de bruit aléatoire
    test_noise = torch.randn(16, latent_dim, 1, 1, device=device)  
    # Alternatives :
    # - torch.normal(mean, std, size=(...), device=device) pour spécifier une moyenne et un écart type personnalisés
    # - torch.empty(...).uniform_(-1, 1) pour générer un bruit uniforme
    # - Utiliser une seed fixe via `torch.manual_seed(...)` pour résultats reproductibles

    # Génère des images à partir du générateur rechargé
    reloaded_images = reloaded_generator(test_noise).detach().cpu()  
    # Alternatives :
    # - Ne pas appeler detach() si vous souhaitez conserver la trace des gradients
    # - Utiliser `.clone()` pour éviter d'altérer les données originales

    # Remet les pixels dans la plage [0, 1] (si normalisé en [-1, 1] avant)
    reloaded_images = reloaded_images * 0.5 + 0.5  
    # Alternatives :
    # - reloaded_images.add(1).div(2) (même effet, différente syntaxe)
    # - torchvision.transforms.Normalize(...) avec des valeurs inverses pour "dénormaliser"

# Affiche les images générées sous forme de grille
plt.figure(figsize=(4, 4))  
# Alternatives :
# - figsize=(8, 8) ou plus pour affichage haute résolution
# - fig = plt.subplots(...) si vous voulez un contrôle plus précis

plt.axis("off")  
# Alternatives :
# - plt.axis("on") pour afficher les axes (utile pour debug)
# - plt.grid(True) si vous voulez des repères

plt.title("Images from Reloaded Generator")  
# Alternatives :
# - Ajouter des infos contextuelles dynamiques : f"Images - Epoch {epoch} - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
# - Utiliser plusieurs sous-titres avec `plt.suptitle(...)` si vous avez plusieurs figures

# Affiche la grille dans le bon format pour matplotlib
plt.imshow(np.transpose(make_grid(reloaded_images, padding=2, normalize=True), (1, 2, 0)))  
# Alternatives :
# - torchvision.utils.save_image(reloaded_images, "generated_grid.png", nrow=4, normalize=True) pour sauvegarder au lieu d'afficher
# - Utiliser PIL.Image.fromarray(...) pour manipuler l’image différemment

plt.show()  
# Alternatives :
# - plt.savefig("reloaded_generator_output.png") pour enregistrer l'image sur disque
# - plt.pause(0.001) dans des environnements interactifs


In [ ]:
import torch  # PyTorch pour opérations ML & GPU; alternative: TensorFlow (tf), JAX (jax.numpy)
import torch.nn as nn  # Modules NN (layers); alternative: torch.nn.functional pour fonctions stateless, ou frameworks comme Keras
import torch.optim as optim  # Optimiseurs; alternatives: SGD, RMSprop, AdamW, AdaGrad, LARS
from torchvision import datasets, transforms, utils  # Datasets + transformations + utils; alternatives: custom datasets, albumentations, PIL.Image
from torch.utils.data import DataLoader  # Chargement batch data; alternatives: DataLoader avec sampler personnalisé, torch.utils.data.DataLoader avec multiprocessing
import matplotlib.pyplot as plt  # Visualisation images; alternatives: seaborn, plotly, PIL.Image.show(), OpenCV (cv2.imshow)
import numpy as np  # Calcul numérique; alternatives: torch.Tensor pour tout, pandas pour tableaux, numba pour optimisation

import os  # Gestion fichiers/chemins; alternative: pathlib (plus moderne et orienté objets)

# Choix device : GPU si disponible sinon CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
# Alternatives: forcer CPU avec torch.device("cpu"), ou définir device spécifique cuda:0, cuda:1 si plusieurs GPUs

print(f"Using device: {device}")  
# Alternative: logging.info(f"Using device: {device}") pour journalisation, ou print(colorama pour couleur)

# Chemins fichiers pth (uploadés dans dossier Kaggle Input)
GENERATOR_PATH = "/kaggle/working/generator_epoch_25.pth" 
# Alternative: "./generator.pth" pour chemin local, ou pathlib.Path("...") pour manipulation objet chemin

DISCRIMINATOR_PATH = "/kaggle/working/discriminator_epoch_25.pth"
# Alternative: utiliser os.path.join pour concaténation robuste ou variables d'environnement

BATCH_SIZE = 64  
# Alternative: ajuster dynamiquement selon mémoire GPU (torch.cuda.get_device_properties), ou tester 32,128 selon vitesse/mémoire

LATENT_DIM = 100  
# Alternative: augmenter latent_dim pour plus de richesse, ou diminuer pour plus rapide/facile à entraîner

IMG_SIZE = 32  
# Alternative: dataset plus grand (64, 128), ou resize dynamique avec transforms.RandomResizedCrop

IMG_CHANNELS = 3  
# Alternative: 1 pour grayscale (ex: MNIST), 4 si RGBA ou image multispectrale

# --- Generator ---
class Generator(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM, img_channels=IMG_CHANNELS, img_size=IMG_SIZE):
        super(Generator, self).__init__()
        self.init_size = img_size // 4  
        # Alternative: diviser par 8 ou 16 pour architectures plus profondes; ou ne pas diviser pour taille originale

        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))  
        # Alternative: ajouter dropout, batchnorm, layernorm, ou plusieurs couches linéaires (MLP profond)

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),  
            # Alternatives: InstanceNorm2d, GroupNorm, LayerNorm, ou pas de normalisation (parfois bénéfique)

            nn.Upsample(scale_factor=2),  
            # Alternatives: ConvTranspose2d pour upsampling appris, PixelShuffle, ou interpolation bilinéaire

            nn.Conv2d(128, 128, 3, stride=1, padding=1),  
            # Alternatives: kernels 5x5, depthwise separable conv (MobileNet style), dilated convolutions

            nn.BatchNorm2d(128, 0.8),  
            # Alternative: momentum différent, ou remplacer batchnorm par dropout spatial

            nn.ReLU(inplace=True),  
            # Alternatives: LeakyReLU(negative_slope=0.2), PReLU (paramétrique), ELU, GELU, SELU pour self-normalizing

            nn.Upsample(scale_factor=2),  
            # Alternatives: ConvTranspose2d avec stride=2, ou resize + conv

            nn.Conv2d(128, 64, 3, stride=1, padding=1),  
            # Alternative: groupes, profondeur separable, normalisation ou pas

            nn.BatchNorm2d(64, 0.8),  
            # Alternative: LayerNorm, InstanceNorm, ou sans normalisation

            nn.ReLU(inplace=True),  
            # Alternative: same as above activations

            nn.Conv2d(64, img_channels, 3, stride=1, padding=1),  
            # Alternative: 1 canal grayscale, 4 canaux RGBA, ou même channels pour style transfer

            nn.Tanh()  
            # Alternative: Sigmoid (sortie 0-1) si normalisation dataset différente, ou pas d’activation finale pour GAN Wasserstein
        )

    def forward(self, z):
        out = self.l1(z)  
        # Alternative: activation (ex: LeakyReLU) après linéaire pour mieux entraîner

        out = out.view(out.size(0), 128, self.init_size, self.init_size)  
        # Alternative: torch.reshape(out, (batch_size, 128, init_size, init_size)) ou .permute si réarrangement nécessaire

        img = self.conv_blocks(out)  
        # Alternative: plus de blocs, blocs résiduels, attention (self-attention GAN)

        return img

# --- Discriminator ---
class Discriminator(nn.Module):
    def __init__(self, img_channels=IMG_CHANNELS, img_size=IMG_SIZE):
        super(Discriminator, self).__init__()

        def disc_block(in_filters, out_filters, bn=True):
            layers = [
                nn.Conv2d(in_filters, out_filters, 3, 2, 1),  
                # Alternative: stride=1 + MaxPool2d, dilated conv, spectral normalization

                nn.LeakyReLU(0.2, inplace=True),  
                # Alternative: ReLU, PReLU, ELU, SELU

                nn.Dropout2d(0.25)  
                # Alternative: SpatialDropout, Dropout classique, ou pas de dropout pour GAN moins régularisés
            ]
            if bn:
                layers.append(nn.BatchNorm2d(out_filters, 0.8))  
                # Alternative: InstanceNorm2d, LayerNorm, SpectralNorm (très utile en GANs pour stabilité)

            return layers

        self.model = nn.Sequential(
            *disc_block(img_channels, 16, bn=False),  
            # Alternative: mettre batchnorm aussi ici

            *disc_block(16, 32),
            *disc_block(32, 64),
            *disc_block(64, 128),
            # Alternative: plus ou moins de blocs selon complexité/dataset
        )

        ds_size = img_size // 2**4  
        # Alternative: changer profondeur pour taille finale différente

        self.adv_layer = nn.Sequential(
            nn.Linear(128 * ds_size * ds_size, 1),  
            # Alternative: convolution 1x1 + GAP (global average pooling)

            nn.Sigmoid()  
            # Alternative: pas d’activation + BCEWithLogitsLoss (plus stable), ou sorties logits pour Wasserstein GAN
        )

    def forward(self, img):
        out = self.model(img)  
        # Alternative: ajouter bruit gaussien, ou labels smoothing dans forward

        out = out.view(out.size(0), -1)  
        # Alternative: torch.flatten(out, start_dim=1)

        validity = self.adv_layer(out)  
        # Alternative: multiple sorties (multi-tâches), ou features pour losses additionnelles

        return validity

# --- Instanciation et chargement ---
generator = Generator().to(device)  
# Alternative: charger modèle scripté torch.jit.load(), ou DataParallel pour multi-GPU

discriminator = Discriminator().to(device)  
# Alternative: charger partiellement (strict=False), ou avec try-except pour gérer erreurs

print(f"Loading generator from {GENERATOR_PATH}")
generator.load_state_dict(torch.load(GENERATOR_PATH, map_location=device))  
# Alternative: torch.jit.load pour modèle complet (archi + poids)

print(f"Loading discriminator from {DISCRIMINATOR_PATH}")
discriminator.load_state_dict(torch.load(DISCRIMINATOR_PATH, map_location=device))  
# Alternative: charger avec strict=False si changement architecture léger

# --- Dataset CIFAR-10 ---
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  
    # Alternative: transforms.CenterCrop, RandomCrop, RandomResizedCrop pour augmentation

    transforms.ToTensor(),  
    # Alternative: transforms.ToPILImage inverse, custom transforms

    transforms.Normalize([0.5]*3, [0.5]*3),  
    # Alternative: normalisation selon stats CIFAR10 (mean/std), ou pas de normalisation
])

train_dataset = datasets.CIFAR10(root="/kaggle/working/data", train=True, download=True, transform=transform)  
# Alternative: STL10, CelebA, custom datasets, ou subset spécifique

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)  
# Alternative: shuffle=False, sampler personnalisé, num_workers>0 pour paralléliser le chargement

# --- Loss & Optimizers ---
adversarial_loss = nn.BCELoss()  
# Alternative: nn.BCEWithLogitsLoss, hinge loss, Wasserstein loss (WGAN-GP)

optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))  
# Alternative: RMSprop, SGD avec momentum, AdamW, Adam avec lr scheduler

optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))  
# Alternative: lr différent du générateur, scheduler de lr

# --- Training Loop ---
EPOCHS = 10  
# Alternative: early stopping, augmenter/diminuer epochs, ou scheduler pour lr

for epoch in range(EPOCHS):
    generator.train()  
    # Alternative: eval() pour freeze, ou mixed precision training (torch.cuda.amp.autocast)

    discriminator.train()  
    # Alternative: eval() si on veut figer discriminateur temporairement

    for batch_idx, (imgs, _) in enumerate(train_loader):
        batch_size = imgs.size(0)  
        # Alternative: batch fixe ou batch dynamique

        real_imgs = imgs.to(device)  
        # Alternative: mixed precision, augmenter bruit (Gaussian noise)

        valid = torch.ones(batch_size, 1, device=device)  
        # Alternative: label smoothing (0.9), label flipping (invert labels)

        fake = torch.zeros(batch_size, 1, device=device)  
        # Alternative: same as valid, label noise

        # Train Discriminator
        optimizer_D.zero_grad()  
        # Alternative: optimizer_D.zero_grad(set_to_none=True) pour performance mémoire

        real_pred = discriminator(real_imgs)  
        # Alternative: ajouter bruit ou augmentation

        real_loss = adversarial_loss(real_pred, valid)  
        # Alternative: Wasserstein loss ou hinge loss

        z = torch.randn(batch_size, LATENT_DIM, device=device)  
        # Alternative: uniform noise, bruit fixe, bruit conditionnel

        fake_imgs = generator(z)  
        # Alternative: ajouter bruit input ou layers dropout

        fake_pred = discriminator(fake_imgs.detach())  
        # Alternative: pas de detach() pour backprop conjoint

        fake_loss = adversarial_loss(fake_pred, fake)  
        # Alternative: weighted loss, hinge loss

        d_loss = (real_loss + fake_loss) / 2  
        # Alternative: pondération différente, ou loss plus complexe (feature matching)

        d_loss.backward()  
        # Alternative: gradient clipping

        optimizer_D.step()  
        # Alternative: scheduler.step(), ou optimizer alternatif

        # Train Generator
        optimizer_G.zero_grad()  
        # Alternative: set_to_none=True, ou accumulate gradients

        gen_pred = discriminator(fake_imgs)  
        # Alternative: ajouter bruit pour régularisation

        g_loss = adversarial_loss(gen_pred, valid)  
        # Alternative: feature matching, perceptual loss, Wasserstein loss

        g_loss.backward()  
        # Alternative: gradient accumulation, mixed precision

        optimizer_G.step()  
        # Alternative: optimiser moins fréquemment (1x every n itérations)

        if batch_idx % 200 == 0:
            print(f"[Epoch {epoch+1}/{EPOCHS}] Batch {batch_idx}/{len(train_loader)} D_loss: {d_loss.item():.4f} G_loss: {g_loss.item():.4f}")
            # Alternative: log dans tensorboard, wandb, ou fichier

# --- Génération finale ---
generator.eval()  
# Alternative: utiliser torch.no_grad() ou torch.inference_mode() pour meilleur perf

with torch.no_grad():
    z = torch.randn(16, LATENT_DIM, device=device)  
    # Alternative: seed fixé (torch.manual_seed) pour reproductibilité

    gen_imgs = generator(z).cpu()  
    # Alternative: garder sur GPU pour post-traitement

    gen_imgs = (gen_imgs + 1) / 2  
    # Alternative: min-max normalization, ou pas de normalisation si Tanh absente

grid = utils.make_grid(gen_imgs, nrow=4)  
# Alternative: torchvision.utils.save_image pour sauvegarder, ou montage personnalisé

plt.figure(figsize=(8,8))  
# Alternative: plt.subplots() pour plus de contrôle

plt.axis("off")  
# Alternative: plt.axis("on") pour debug axes

plt.title("Images générées après fine-tuning CIFAR-10")  
# Alternative: inclure timestamp, epoch, loss

plt.imshow(np.transpose(grid.numpy(), (1, 2, 0)))  
# Alternative: plt.imshow(grid.permute(1,2,0)) si tensor, ou PIL.Image.fromarray

plt.show()  
# Alternative: plt.savefig("output.png") pour sauvegarder



In [ ]:
# ==================================
# SETUP : LIBRAIRIES ET HYPERPARAMÈTRES GLOBAUX
# ==================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import numpy as np
import os
from copy import deepcopy

# --- Création des dossiers pour sauvegarder les résultats ---
os.makedirs("mnist_images", exist_ok=True)
os.makedirs("cifar10_images", exist_ok=True)
os.makedirs("models", exist_ok=True)


# --- Hyperparamètres ---
batch_size = 128
image_size = 64
latent_dim = 100
lr_pretrain = 0.0002  # Taux d'apprentissage pour le pré-entraînement
lr_finetune = 0.0001 # Taux d'apprentissage plus faible pour le fine-tuning
betas = (0.5, 0.999) # Bêtas pour l'optimiseur Adam, éprouvés pour les GANs
epochs_pretrain = 15 # Nombre d'époques pour MNIST (pas besoin de beaucoup pour des features de base)
epochs_finetune = 25 # Nombre d'époques pour le fine-tuning sur CIFAR-10

# --- Configuration du device (GPU si disponible) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du device: {device}")

In [ ]:
# =======================================================
# PHASE 1 : PRÉ-ENTRAÎNEMENT DU GAN SUR MNIST (SOURCE)
# =======================================================

print("\n--- DÉBUT DE LA PHASE 1 : PRÉ-ENTRAÎNEMENT SUR MNIST ---")

# --- 1.1. Préparation des données MNIST ---
image_channels_mnist = 1
transform_mnist = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # Normalisation pour des images monochromes
])

mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
dataloader_mnist = DataLoader(mnist_dataset, batch_size=batch_size, shuffle=True)


# --- 1.2. Définition des modèles (Générateur et Discriminateur) ---
# NOTE : Ces classes sont identiques à celles fournies, mais nous les redéfinissons ici pour la clarté.

class Generator(nn.Module):
    def __init__(self, latent_dim, channels):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False), nn.BatchNorm2d(512), nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False), nn.BatchNorm2d(256), nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False), nn.BatchNorm2d(128), nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False), nn.BatchNorm2d(64), nn.ReLU(True),
            nn.ConvTranspose2d(64, channels, 4, 2, 1, bias=False), nn.Tanh()
        )
    def forward(self, z): return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(channels, 64, 4, 2, 1, bias=False), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False), nn.BatchNorm2d(128), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), nn.BatchNorm2d(256), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False), nn.BatchNorm2d(512), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False), nn.Sigmoid()
        )
    def forward(self, img): return self.model(img).view(-1)

# Fonction d'initialisation des poids (pratique standard pour les DCGAN)
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1: nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1: nn.init.normal_(m.weight.data, 1.0, 0.02); nn.init.constant_(m.bias.data, 0)

# --- 1.3. Initialisation et entraînement ---
generator_mnist = Generator(latent_dim, image_channels_mnist).to(device)
discriminator_mnist = Discriminator(image_channels_mnist).to(device)
generator_mnist.apply(weights_init)
discriminator_mnist.apply(weights_init)

criterion = nn.BCELoss()
optimizer_G_mnist = optim.Adam(generator_mnist.parameters(), lr=lr_pretrain, betas=betas)
optimizer_D_mnist = optim.Adam(discriminator_mnist.parameters(), lr=lr_pretrain, betas=betas)

fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

print("Début de l'entraînement sur MNIST...")
for epoch in range(epochs_pretrain):
    for i, (real_images, _) in enumerate(dataloader_mnist):
        real_images = real_images.to(device)
        real_labels = torch.ones(real_images.size(0), device=device)
        fake_labels = torch.zeros(real_images.size(0), device=device)

        # --- Entraînement du Discriminateur ---
        optimizer_D_mnist.zero_grad()
        # Perte sur les images réelles
        d_output_real = discriminator_mnist(real_images)
        errD_real = criterion(d_output_real, real_labels)
        errD_real.backward()
        # Perte sur les images générées
        noise = torch.randn(real_images.size(0), latent_dim, 1, 1, device=device)
        fake_images = generator_mnist(noise)
        d_output_fake = discriminator_mnist(fake_images.detach())
        errD_fake = criterion(d_output_fake, fake_labels)
        errD_fake.backward()
        errD = errD_real + errD_fake
        optimizer_D_mnist.step()

        # --- Entraînement du Générateur ---
        optimizer_G_mnist.zero_grad()
        d_output_on_fake = discriminator_mnist(fake_images)
        errG = criterion(d_output_on_fake, real_labels)
        errG.backward()
        optimizer_G_mnist.step()

    print(f"[Epoch MNIST {epoch+1}/{epochs_pretrain}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}")
    with torch.no_grad():
        fake_samples = generator_mnist(fixed_noise).detach().cpu()
    save_image(fake_samples, f"mnist_images/mnist_fake_epoch_{epoch+1}.png", normalize=True)

# Sauvegarde des modèles pré-entraînés
torch.save(generator_mnist.state_dict(), "models/generator_mnist.pth")
torch.save(discriminator_mnist.state_dict(), "models/discriminator_mnist.pth")
print("--- FIN DE LA PHASE 1 : Modèles MNIST pré-entraînés et sauvegardés. ---")

In [ ]:
# =========================================================================
# PHASE 2 : TRANSFERT D'APPRENTISSAGE ET FINE-TUNING SUR CIFAR-10 (CIBLE)
# =========================================================================

print("\n--- DÉBUT DE LA PHASE 2 : TRANSFERT ET FINE-TUNING SUR CIFAR-10 ---")

# --- 2.1. Préparation des données CIFAR-10 ---
image_channels_cifar = 3 # CIFAR-10 a 3 canaux de couleur (RGB)
transform_cifar = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # Normalisation pour 3 canaux
])

cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
dataloader_cifar = DataLoader(cifar_dataset, batch_size=batch_size, shuffle=True)

# Visualisation d'un batch d'images CIFAR-10
real_batch_cifar = next(iter(dataloader_cifar))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Batch d'images réelles de CIFAR-10")
grid = make_grid(real_batch_cifar[0][:64] * 0.5 + 0.5, padding=2, normalize=True)
plt.imshow(np.transpose(grid.cpu(), (1, 2, 0)))
plt.show()


# --- 2.2. Création des nouveaux modèles et transfert des poids ---

# Nous créons de nouvelles instances de modèles pour CIFAR-10
# Notez le changement du nombre de canaux
generator_cifar = Generator(latent_dim, image_channels_cifar).to(device)
discriminator_cifar = Discriminator(image_channels_cifar).to(device)

# Chargement des poids pré-entraînés de MNIST
gen_mnist_weights = torch.load("models/generator_mnist.pth", map_location=device)
disc_mnist_weights = torch.load("models/discriminator_mnist.pth", map_location=device)

# *** ÉTAPE CRUCIALE : Transfert des poids compatibles ***
# La dernière couche du générateur et la première du discriminateur ont des dimensions
# différentes (à cause du nombre de canaux). Nous les ignorons lors du chargement.
# Le reste des poids (les couches profondes) sera transféré.

# Pour le Générateur
# La couche finale "model.8.weight" (ConvTranspose2d) ne sera pas chargée
generator_cifar.load_state_dict(gen_mnist_weights, strict=False)
print("Poids du générateur MNIST transférés (sauf la couche de sortie).")

# Pour le Discriminateur
# La couche initiale "model.0.weight" (Conv2d) ne sera pas chargée
discriminator_cifar.load_state_dict(disc_mnist_weights, strict=False)
print("Poids du discriminateur MNIST transférés (sauf la couche d'entrée).")


# --- 2.3. Gel (Freeze) des couches transférées ---
# Nous gelons les 3 premiers blocs de chaque modèle. Seules les couches
# plus profondes (spécifiques à la tâche) seront entraînées au début.

# Fonction utilitaire pour vérifier l'état des couches
def print_trainable_status(model, model_name):
    print(f"\nStatut des paramètres pour {model_name}:")
    for name, param in model.named_parameters():
        print(f"{name:<40} Entraînable: {param.requires_grad}")

# Geler les couches
layers_to_freeze_g = 6 # Les 3 premiers blocs (ConvT, BatchNorm)
layers_to_freeze_d = 6 # Les 3 premiers blocs (Conv, BatchNorm)

for i, (name, param) in enumerate(generator_cifar.named_parameters()):
    if i < layers_to_freeze_g:
        param.requires_grad = False

for i, (name, param) in enumerate(discriminator_cifar.named_parameters()):
    if i < layers_to_freeze_d:
        param.requires_grad = False

print_trainable_status(generator_cifar, "Générateur CIFAR-10")
print_trainable_status(discriminator_cifar, "Discriminateur CIFAR-10")

# --- 2.4. Fine-Tuning sur CIFAR-10 ---
# On crée des optimiseurs qui ne mettront à jour que les poids "dégelés" (requires_grad=True)
params_g_finetune = filter(lambda p: p.requires_grad, generator_cifar.parameters())
params_d_finetune = filter(lambda p: p.requires_grad, discriminator_cifar.parameters())

optimizer_G_cifar = optim.Adam(params_g_finetune, lr=lr_finetune, betas=betas)
optimizer_D_cifar = optim.Adam(params_d_finetune, lr=lr_finetune, betas=betas)

print("\nDébut du Fine-Tuning sur CIFAR-10...")
for epoch in range(epochs_finetune):
    for i, (real_images, _) in enumerate(dataloader_cifar):
        real_images = real_images.to(device)
        real_labels = torch.ones(real_images.size(0), device=device)
        fake_labels = torch.zeros(real_images.size(0), device=device)

        # --- Entraînement du Discriminateur ---
        optimizer_D_cifar.zero_grad()
        d_output_real = discriminator_cifar(real_images)
        errD_real = criterion(d_output_real, real_labels)
        errD_real.backward()

        noise = torch.randn(real_images.size(0), latent_dim, 1, 1, device=device)
        fake_images = generator_cifar(noise)
        d_output_fake = discriminator_cifar(fake_images.detach())
        errD_fake = criterion(d_output_fake, fake_labels)
        errD_fake.backward()
        errD = errD_real + errD_fake
        optimizer_D_cifar.step()

        # --- Entraînement du Générateur ---
        optimizer_G_cifar.zero_grad()
        d_output_on_fake = discriminator_cifar(fake_images)
        errG = criterion(d_output_on_fake, real_labels)
        errG.backward()
        optimizer_G_cifar.step()

    print(f"[Epoch CIFAR-10 {epoch+1}/{epochs_finetune}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}")
    with torch.no_grad():
        fake_samples = generator_cifar(fixed_noise).detach().cpu()
    save_image(fake_samples, f"cifar10_images/cifar10_fake_epoch_{epoch+1}.png", normalize=True)


print("--- FIN DE LA PHASE 2 : Fine-tuning terminé. ---")


# --- 2.5. Génération et Visualisation finale ---
generator_cifar.eval()
with torch.no_grad():
    final_noise = torch.randn(64, latent_dim, 1, 1, device=device)
    generated_images = generator_cifar(final_noise).detach().cpu()

plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("Images CIFAR-10 finales générées par Transfer Learning")
plt.imshow(np.transpose(make_grid(generated_images, padding=2, normalize=True), (1, 2, 0)))
plt.show()

In [ ]:
# =========================================================================
# PHASE 2 : TRANSFERT D'APPRENTISSAGE ET FINE-TUNING SUR CIFAR-10 (CIBLE)
# =========================================================================

print("\n--- DÉBUT DE LA PHASE 2 : TRANSFERT ET FINE-TUNING SUR CIFAR-10 ---")

# --- 2.1. Préparation des données CIFAR-10 ---
image_channels_cifar = 3 # CIFAR-10 a 3 canaux de couleur (RGB)
transform_cifar = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # Normalisation pour 3 canaux
])

cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
dataloader_cifar = DataLoader(cifar_dataset, batch_size=batch_size, shuffle=True)

# Visualisation d'un batch d'images CIFAR-10
real_batch_cifar = next(iter(dataloader_cifar))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Batch d'images réelles de CIFAR-10")
grid = make_grid(real_batch_cifar[0][:64] * 0.5 + 0.5, padding=2, normalize=True)
plt.imshow(np.transpose(grid.cpu(), (1, 2, 0)))
plt.show()

# --- 2.2. (CORRIGÉ) Création des nouveaux modèles et transfert des poids ---
generator_cifar = Generator(latent_dim, image_channels_cifar).to(device)
discriminator_cifar = Discriminator(image_channels_cifar).to(device)

gen_mnist_weights = torch.load("models/generator_mnist.pth", map_location=device)
disc_mnist_weights = torch.load("models/discriminator_mnist.pth", map_location=device)

# Filtrage pour le Générateur
gen_cifar_dict = generator_cifar.state_dict()
pretrained_gen_dict = {k: v for k, v in gen_mnist_weights.items() if k in gen_cifar_dict and gen_cifar_dict[k].size() == v.size()}
gen_cifar_dict.update(pretrained_gen_dict)
generator_cifar.load_state_dict(gen_cifar_dict)
print(f"Transfert de {len(pretrained_gen_dict)}/{len(gen_cifar_dict)} couches du Générateur MNIST vers CIFAR-10.")

# Filtrage pour le Discriminateur
disc_cifar_dict = discriminator_cifar.state_dict()
pretrained_disc_dict = {k: v for k, v in disc_mnist_weights.items() if k in disc_cifar_dict and disc_cifar_dict[k].size() == v.size()}
disc_cifar_dict.update(pretrained_disc_dict)
discriminator_cifar.load_state_dict(disc_cifar_dict)
print(f"Transfert de {len(pretrained_disc_dict)}/{len(disc_cifar_dict)} couches du Discriminateur MNIST vers CIFAR-10.")


# --- 2.3. Gel (Freeze) des couches transférées ---
# Cette partie est correcte et très importante pour la stabilité du fine-tuning.
def print_trainable_status(model, model_name):
    print(f"\nStatut des paramètres pour {model_name}:")
    total_params = 0
    trainable_params = 0
    for name, param in model.named_parameters():
        total_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
        print(f"{name:<40} Entraînable: {param.requires_grad}")
    print(f"Paramètres entraînables : {trainable_params} / {total_params} ({100 * trainable_params / total_params:.2f}%)")


# Nous gelons les couches qui ont été chargées depuis MNIST pour ne pas les "oublier" trop vite.
# On ne fine-tune que les dernières couches (celles qui n'ont pas été chargées + les plus profondes)
layers_to_freeze_g = 8 # Gelons les 4 premiers blocs
layers_to_freeze_d = 8 # Gelons les 4 premiers blocs

for i, (name, param) in enumerate(generator_cifar.named_parameters()):
    if name in pretrained_gen_dict: # Une façon plus robuste de geler que de compter
        param.requires_grad = False

for i, (name, param) in enumerate(discriminator_cifar.named_parameters()):
     if name in pretrained_disc_dict:
        param.requires_grad = False

print_trainable_status(generator_cifar, "Générateur CIFAR-10")
print_trainable_status(discriminator_cifar, "Discriminateur CIFAR-10")


# --- 2.4. Fine-Tuning sur CIFAR-10 ---
# Le reste de votre code est correct.
params_g_finetune = filter(lambda p: p.requires_grad, generator_cifar.parameters())
params_d_finetune = filter(lambda p: p.requires_grad, discriminator_cifar.parameters())

optimizer_G_cifar = optim.Adam(params_g_finetune, lr=lr_finetune, betas=betas)
optimizer_D_cifar = optim.Adam(params_d_finetune, lr=lr_finetune, betas=betas)

print("\nDébut du Fine-Tuning sur CIFAR-10...")
# ... la boucle d'entraînement reste inchangée ...
for epoch in range(epochs_finetune):
    for i, (real_images, _) in enumerate(dataloader_cifar):
        real_images = real_images.to(device)
        real_labels = torch.ones(real_images.size(0), device=device)
        fake_labels = torch.zeros(real_images.size(0), device=device)

        # Entraînement du Discriminateur
        optimizer_D_cifar.zero_grad()
        d_output_real = discriminator_cifar(real_images)
        errD_real = criterion(d_output_real, real_labels)
        errD_real.backward()
        noise = torch.randn(real_images.size(0), latent_dim, 1, 1, device=device)
        fake_images = generator_cifar(noise)
        d_output_fake = discriminator_cifar(fake_images.detach())
        errD_fake = criterion(d_output_fake, fake_labels)
        errD_fake.backward()
        errD = errD_real + errD_fake
        optimizer_D_cifar.step()

        # Entraînement du Générateur
        optimizer_G_cifar.zero_grad()
        d_output_on_fake = discriminator_cifar(fake_images)
        errG = criterion(d_output_on_fake, real_labels)
        errG.backward()
        optimizer_G_cifar.step()

    print(f"[Epoch CIFAR-10 {epoch+1}/{epochs_finetune}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}")
    with torch.no_grad():
        fake_samples = generator_cifar(fixed_noise).detach().cpu()
    save_image(fake_samples, f"cifar10_images/cifar10_fake_epoch_{epoch+1}.png", normalize=True)

print("--- FIN DE LA PHASE 2 : Fine-tuning terminé. ---")


# --- 2.5. Génération et Visualisation finale ---
# ... cette section reste inchangée ...
generator_cifar.eval()
with torch.no_grad():
    final_noise = torch.randn(64, latent_dim, 1, 1, device=device)
    generated_images = generator_cifar(final_noise).detach().cpu()

plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("Images CIFAR-10 finales générées par Transfer Learning")
plt.imshow(np.transpose(make_grid(generated_images, padding=2, normalize=True), (1, 2, 0)))
plt.show()

In [ ]:
# =========================================================================
# PHASE 2 : TRANSFERT D'APPRENTISSAGE ET FINE-TUNING SUR CIFAR-10 (CIBLE)
# =========================================================================
# TECHNIQUE: Organisation modulaire du code avec séparateurs visuels
# ALTERNATIVES: Utiliser des classes, des modules séparés, ou des notebooks Jupyter
# ENFANT: C'est comme mettre une pancarte pour dire "Ici commence la partie 2 de notre programme"

print("\n--- DÉBUT DE LA PHASE 2 : TRANSFERT ET FINE-TUNING SUR CIFAR-10 ---")
# TECHNIQUE: Logging informatif avec séparateurs visuels (\n pour nouvelle ligne)
# ALTERNATIVES: logging.info(), tqdm.write(), print avec timestamp, wandb.log()
# ENFANT: On dit à l'ordinateur d'écrire un message pour nous dire où on en est

# --- 2.1. Préparation des données CIFAR-10 ---
image_channels_cifar = 3 # CIFAR-10 a 3 canaux de couleur (RGB)
# TECHNIQUE: Constante pour définir le nombre de canaux couleur (RGB = Rouge, Vert, Bleu)
# ALTERNATIVES: Utiliser enum.Enum, dataclass, config.yaml, argparse
# ENFANT: On dit à l'ordinateur que nos images ont 3 couleurs : rouge, vert et bleu (comme un arc-en-ciel)

transform_cifar = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # Normalisation pour 3 canaux
])
# TECHNIQUE: Pipeline de transformation avec Compose pour chaîner les opérations
# ALTERNATIVES: transforms.v2, albumentations, kornia, transforms personnalisées, tf.data
# ENFANT: C'est comme une recette : d'abord on redimensionne l'image, puis on la transforme en nombres, puis on l'ajuste

cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
# TECHNIQUE: Chargement automatique du dataset avec téléchargement et transformations
# ALTERNATIVES: torchdata, datasets (HuggingFace), tf.data, chargement manuel avec PIL/cv2
# ENFANT: On demande à l'ordinateur de télécharger des milliers d'images d'animaux et d'objets

dataloader_cifar = DataLoader(cifar_dataset, batch_size=batch_size, shuffle=True)
# TECHNIQUE: DataLoader pour traitement par batch avec mélange aléatoire
# ALTERNATIVES: torch.utils.data.distributed, tf.data.Dataset, ray.data, custom iterators
# ENFANT: C'est comme prendre une poignée d'images au hasard dans un grand sac pour les montrer à notre IA

# Visualisation d'un batch d'images CIFAR-10
real_batch_cifar = next(iter(dataloader_cifar))
# TECHNIQUE: Extraction du premier batch avec next() et iter() pour inspection
# ALTERNATIVES: enumerate(), for loop avec break, dataset[indices], random sampling
# ENFANT: On prend la première poignée d'images pour les regarder

plt.figure(figsize=(8, 8))
# TECHNIQUE: Création d'une figure matplotlib avec taille spécifiée
# ALTERNATIVES: plt.subplots(), fig.add_subplot(), seaborn, plotly, cv2.imshow()
# ENFANT: On prépare une feuille de papier carrée pour dessiner dessus

plt.axis("off")
# TECHNIQUE: Suppression des axes pour affichage propre des images
# ALTERNATIVES: plt.xticks([]), plt.yticks([]), sns.despine(), ax.axis('off')
# ENFANT: On enlève les lignes et les chiffres autour de notre dessin pour que ce soit plus joli

plt.title("Batch d'images réelles de CIFAR-10")
# TECHNIQUE: Ajout d'un titre descriptif à la visualisation
# ALTERNATIVES: ax.set_title(), fig.suptitle(), plt.text(), annotations personnalisées
# ENFANT: On écrit un titre au-dessus de nos images comme dans un livre

grid = make_grid(real_batch_cifar[0][:64] * 0.5 + 0.5, padding=2, normalize=True)
# TECHNIQUE: Création d'une grille d'images avec normalisation [-1,1] -> [0,1]
# ALTERNATIVES: torchvision.utils.save_image(), custom grid, PIL.Image.thumbnail(), cv2.hconcat()
# ENFANT: On arrange nos images en carré comme des cartes sur une table, et on les rend plus jolies

plt.imshow(np.transpose(grid.cpu(), (1, 2, 0)))
# TECHNIQUE: Transposition des dimensions (C,H,W) -> (H,W,C) pour matplotlib
# ALTERNATIVES: einops.rearrange(), torch.permute(), PIL conversion, cv2.cvtColor()
# ENFANT: On change l'ordre des couleurs pour que l'ordinateur puisse bien afficher l'image

plt.show()
# TECHNIQUE: Affichage de la figure dans l'interface
# ALTERNATIVES: plt.savefig(), plt.display(), wandb.log(), tensorboard
# ENFANT: On montre notre belle image à l'écran

# --- 2.2. (CORRIGÉ) Création des nouveaux modèles et transfert des poids ---
generator_cifar = Generator(latent_dim, image_channels_cifar).to(device)
# TECHNIQUE: Instanciation du générateur pour CIFAR-10 avec transfert sur GPU/CPU
# ALTERNATIVES: nn.DataParallel(), model.cuda(), lazy initialization, factory pattern
# ENFANT: On crée un nouveau "dessinateur" spécialisé pour les images en couleur

discriminator_cifar = Discriminator(image_channels_cifar).to(device)
# TECHNIQUE: Instanciation du discriminateur pour CIFAR-10 avec transfert sur GPU/CPU
# ALTERNATIVES: nn.DataParallel(), model.cuda(), lazy initialization, factory pattern
# ENFANT: On crée un nouveau "détective" spécialisé pour reconnaître les vraies images en couleur

gen_mnist_weights = torch.load("models/generator_mnist.pth", map_location=device)
# TECHNIQUE: Chargement des poids sauvegardés avec map_location pour compatibilité GPU/CPU
# ALTERNATIVES: pickle.load(), joblib.load(), safetensors, checkpoint avec metadata
# ENFANT: On récupère la mémoire de notre ancien "dessinateur" qui savait dessiner des chiffres

disc_mnist_weights = torch.load("models/discriminator_mnist.pth", map_location=device)
# TECHNIQUE: Chargement des poids sauvegardés avec map_location pour compatibilité GPU/CPU
# ALTERNATIVES: pickle.load(), joblib.load(), safetensors, checkpoint avec metadata
# ENFANT: On récupère la mémoire de notre ancien "détective" qui savait reconnaître des chiffres

# Filtrage pour le Générateur
gen_cifar_dict = generator_cifar.state_dict()
# TECHNIQUE: Récupération du dictionnaire d'état du modèle (nom_paramètre -> tensor)
# ALTERNATIVES: model.parameters(), model.named_parameters(), custom state management
# ENFANT: On regarde la liste de tous les "muscles" (paramètres) de notre nouveau dessinateur

pretrained_gen_dict = {k: v for k, v in gen_mnist_weights.items() if k in gen_cifar_dict and gen_cifar_dict[k].size() == v.size()}
# TECHNIQUE: Filtrage par compréhension de dictionnaire avec vérification de compatibilité des tailles
# ALTERNATIVES: set intersection, try/except loading, manual layer matching, regex filtering
# ENFANT: On regarde quels "muscles" de l'ancien dessinateur peuvent s'adapter au nouveau (même taille)

gen_cifar_dict.update(pretrained_gen_dict)
# TECHNIQUE: Mise à jour du dictionnaire d'état avec les poids compatibles
# ALTERNATIVES: manual assignment, torch.nn.utils.prune, selective loading
# ENFANT: On donne au nouveau dessinateur les "muscles" qu'on peut récupérer de l'ancien

generator_cifar.load_state_dict(gen_cifar_dict)
# TECHNIQUE: Chargement du state_dict modifié dans le modèle
# ALTERNATIVES: load_state_dict(strict=False), manual parameter assignment, hooks
# ENFANT: Le nouveau dessinateur apprend tout ce qu'il peut de l'ancien

print(f"Transfert de {len(pretrained_gen_dict)}/{len(gen_cifar_dict)} couches du Générateur MNIST vers CIFAR-10.")
# TECHNIQUE: Logging informatif avec f-string pour le debugging
# ALTERNATIVES: logging.info(), wandb.log(), tensorboard, print avec format()
# ENFANT: On compte combien de "muscles" on a pu transférer et on l'écrit

# Filtrage pour le Discriminateur
disc_cifar_dict = discriminator_cifar.state_dict()
# TECHNIQUE: Récupération du dictionnaire d'état du discriminateur
# ALTERNATIVES: model.parameters(), model.named_parameters(), custom state management
# ENFANT: On regarde la liste de tous les "muscles" de notre nouveau détective

pretrained_disc_dict = {k: v for k, v in disc_mnist_weights.items() if k in disc_cifar_dict and disc_cifar_dict[k].size() == v.size()}
# TECHNIQUE: Filtrage par compréhension avec vérification de compatibilité
# ALTERNATIVES: set intersection, try/except loading, manual layer matching
# ENFANT: On regarde quels "muscles" de l'ancien détective peuvent s'adapter au nouveau

disc_cifar_dict.update(pretrained_disc_dict)
# TECHNIQUE: Mise à jour du dictionnaire avec les poids compatibles
# ALTERNATIVES: manual assignment, selective loading, parameter surgery
# ENFANT: On donne au nouveau détective les "muscles" récupérables de l'ancien

discriminator_cifar.load_state_dict(disc_cifar_dict)
# TECHNIQUE: Chargement du state_dict dans le modèle discriminateur
# ALTERNATIVES: load_state_dict(strict=False), manual loading, progressive loading
# ENFANT: Le nouveau détective apprend tout ce qu'il peut de l'ancien

print(f"Transfert de {len(pretrained_disc_dict)}/{len(disc_cifar_dict)} couches du Discriminateur MNIST vers CIFAR-10.")
# TECHNIQUE: Logging informatif du transfert pour le discriminateur
# ALTERNATIVES: logging with levels, structured logging, metrics tracking
# ENFANT: On compte et on écrit combien de "muscles" du détective ont été transférés

# --- 2.3. Gel (Freeze) des couches transférées ---
def print_trainable_status(model, model_name):
    # TECHNIQUE: Fonction utilitaire pour inspection des paramètres entraînables
    # ALTERNATIVES: torchinfo.summary(), model hooks, custom introspection, wandb.watch()
    # ENFANT: On crée une fonction qui nous dit quels "muscles" peuvent encore apprendre
    
    print(f"\nStatut des paramètres pour {model_name}:")
    # TECHNIQUE: Logging avec formatage pour clarté
    # ALTERNATIVES: tabulate, rich.table, pandas.DataFrame, structured output
    # ENFANT: On écrit le nom du modèle qu'on va examiner
    
    total_params = 0
    trainable_params = 0
    # TECHNIQUE: Compteurs pour statistiques des paramètres
    # ALTERNATIVES: sum() avec generator, collections.Counter, numpy operations
    # ENFANT: On prépare deux compteurs : un pour tous les "muscles", un pour ceux qui apprennent
    
    for name, param in model.named_parameters():
        # TECHNIQUE: Itération sur les paramètres nommés du modèle
        # ALTERNATIVES: model.parameters(), recursive parameter extraction, hooks
        # ENFANT: On regarde chaque "muscle" du modèle un par un
        
        total_params += param.numel()
        # TECHNIQUE: Comptage du nombre d'éléments dans le tenseur
        # ALTERNATIVES: param.size().numel(), torch.numel(), manual calculation
        # ENFANT: On compte combien de petites parties a ce "muscle"
        
        if param.requires_grad:
            trainable_params += param.numel()
        # TECHNIQUE: Comptage conditionnel des paramètres entraînables
        # ALTERNATIVES: filter() avec lambda, list comprehension, boolean indexing
        # ENFANT: Si ce "muscle" peut encore apprendre, on l'ajoute à notre compteur spécial
        
        print(f"{name:<40} Entraînable: {param.requires_grad}")
        # TECHNIQUE: Formatage aligné avec padding pour lisibilité
        # ALTERNATIVES: f-string avec format specs, str.format(), tabulate
        # ENFANT: On écrit le nom du "muscle" et si il peut apprendre (Oui/Non)
    
    print(f"Paramètres entraînables : {trainable_params} / {total_params} ({100 * trainable_params / total_params:.2f}%)")
    # TECHNIQUE: Calcul et affichage de statistiques avec formatage décimal
    # ALTERNATIVES: f-string formatting, round(), numpy.round(), percentage calculation libs
    # ENFANT: On calcule et on écrit combien de "muscles" peuvent apprendre sur le total

layers_to_freeze_g = 8 # Gelons les 4 premiers blocs
# TECHNIQUE: Configuration du nombre de couches à geler (commentaire incorrect : 8 ≠ 4)
# ALTERNATIVES: config file, argparse, percentage-based freezing, layer name patterns
# ENFANT: On décide combien de "muscles" du dessinateur on va "endormir" pour qu'ils n'apprennent plus

layers_to_freeze_d = 8 # Gelons les 4 premiers blocs
# TECHNIQUE: Configuration pour le discriminateur (même problème de commentaire)
# ALTERNATIVES: dynamic freezing, progressive unfreezing, layer-wise learning rates
# ENFANT: On décide combien de "muscles" du détective on va "endormir"

for i, (name, param) in enumerate(generator_cifar.named_parameters()):
    if name in pretrained_gen_dict: # Une façon plus robuste de geler que de compter
        param.requires_grad = False
# TECHNIQUE: Gel basé sur la présence dans le dictionnaire pré-entraîné (plus robuste)
# ALTERNATIVES: regex matching, layer indexing, parameter grouping, hooks
# ENFANT: Pour chaque "muscle" du dessinateur, si il vient de l'ancien, on l'endort

for i, (name, param) in enumerate(discriminator_cifar.named_parameters()):
     if name in pretrained_disc_dict:
        param.requires_grad = False
# TECHNIQUE: Gel basé sur la présence dans le dictionnaire (même méthode)
# ALTERNATIVES: functional approach with map(), parameter groups, selective freezing
# ENFANT: Pour chaque "muscle" du détective, si il vient de l'ancien, on l'endort

print_trainable_status(generator_cifar, "Générateur CIFAR-10")
# TECHNIQUE: Appel de fonction pour diagnostic du générateur
# ALTERNATIVES: direct inspection, logging hooks, visualization tools
# ENFANT: On demande à notre fonction de nous dire l'état des "muscles" du dessinateur

print_trainable_status(discriminator_cifar, "Discriminateur CIFAR-10")
# TECHNIQUE: Appel de fonction pour diagnostic du discriminateur
# ALTERNATIVES: combined reporting, comparative analysis, automated testing
# ENFANT: On demande l'état des "muscles" du détective aussi

# --- 2.4. Fine-Tuning sur CIFAR-10 ---
params_g_finetune = filter(lambda p: p.requires_grad, generator_cifar.parameters())
# TECHNIQUE: Filtrage fonctionnel des paramètres entraînables avec lambda
# ALTERNATIVES: list comprehension, [p for p in params if p.requires_grad], parameter groups
# ENFANT: On fait une liste de tous les "muscles" du dessinateur qui peuvent encore apprendre

params_d_finetune = filter(lambda p: p.requires_grad, discriminator_cifar.parameters())
# TECHNIQUE: Même filtrage pour le discriminateur
# ALTERNATIVES: generator expression, manual parameter collection, grouped parameters
# ENFANT: On fait une liste des "muscles" du détective qui peuvent encore apprendre

optimizer_G_cifar = optim.Adam(params_g_finetune, lr=lr_finetune, betas=betas)
# TECHNIQUE: Optimisateur Adam avec paramètres personnalisés et taux d'apprentissage spécifique
# ALTERNATIVES: SGD, RMSprop, AdamW, RAdam, custom schedulers, parameter groups
# ENFANT: On crée un "professeur" spécialisé pour enseigner au dessinateur avec une vitesse d'apprentissage lente

optimizer_D_cifar = optim.Adam(params_d_finetune, lr=lr_finetune, betas=betas)
# TECHNIQUE: Optimisateur séparé pour le discriminateur
# ALTERNATIVES: shared optimizer, different optimizers (TTUR), parameter scheduling
# ENFANT: On crée un "professeur" pour le détective aussi, avec la même vitesse lente

print("\nDébut du Fine-Tuning sur CIFAR-10...")
# TECHNIQUE: Message informatif de début d'entraînement
# ALTERNATIVES: logging with timestamps, progress bars, structured logging
# ENFANT: On annonce qu'on commence l'école spécialisée pour nos deux IA

for epoch in range(epochs_finetune):
    # TECHNIQUE: Boucle d'entraînement par époques
    # ALTERNATIVES: while loops, custom iterators, early stopping, dynamic epochs
    # ENFANT: On répète l'école plusieurs fois (comme plusieurs années scolaires)
    
    for i, (real_images, _) in enumerate(dataloader_cifar):
        # TECHNIQUE: Boucle sur les batches avec énumération et destructuring
        # ALTERNATIVES: while with iterator, batch generators, parallel loading
        # ENFANT: Pour chaque poignée d'images, on va faire une leçon
        
        real_images = real_images.to(device)
        # TECHNIQUE: Transfert des données sur le device (GPU/CPU)
        # ALTERNATIVES: pin_memory, non_blocking transfers, automatic mixed precision
        # ENFANT: On met les images sur le bon ordinateur (rapide ou lent)
        
        real_labels = torch.ones(real_images.size(0), device=device)
        # TECHNIQUE: Création de labels "vrai" (1) pour les vraies images
        # ALTERNATIVES: torch.full(), manual tensor creation, label smoothing
        # ENFANT: On crée des étiquettes qui disent "VRAI" pour les vraies images
        
        fake_labels = torch.zeros(real_images.size(0), device=device)
        # TECHNIQUE: Création de labels "faux" (0) pour les images générées
        # ALTERNATIVES: torch.full(), -torch.ones(), soft labels
        # ENFANT: On crée des étiquettes qui disent "FAUX" pour les fausses images

        # Entraînement du Discriminateur
        optimizer_D_cifar.zero_grad()
        # TECHNIQUE: Remise à zéro des gradients accumulés
        # ALTERNATIVES: set_to_none=True, gradient clipping, gradient accumulation
        # ENFANT: On efface les anciennes leçons du détective pour qu'il puisse apprendre du nouveau
        
        d_output_real = discriminator_cifar(real_images)
        # TECHNIQUE: Forward pass du discriminateur sur vraies images
        # ALTERNATIVES: model.forward(), functional calls, hooks for intermediate outputs
        # ENFANT: Le détective regarde les vraies images et dit ce qu'il pense
        
        errD_real = criterion(d_output_real, real_labels)
        # TECHNIQUE: Calcul de la loss entre prédiction et labels vrais
        # ALTERNATIVES: custom loss functions, weighted losses, focal loss
        # ENFANT: On regarde à quel point le détective s'est trompé sur les vraies images
        
        errD_real.backward()
        # TECHNIQUE: Rétropropagation pour calculer les gradients
        # ALTERNATIVES: manual gradients, retain_graph=True, create_graph=True
        # ENFANT: On calcule comment le détective doit changer pour mieux reconnaître les vraies images
        
        noise = torch.randn(real_images.size(0), latent_dim, 1, 1, device=device)
        # TECHNIQUE: Génération de bruit aléatoire gaussien pour le générateur
        # ALTERNATIVES: uniform noise, learned noise, conditional noise, different distributions
        # ENFANT: On crée du "bruit magique" aléatoire pour que le dessinateur puisse créer des images
        
        fake_images = generator_cifar(noise)
        # TECHNIQUE: Génération d'images à partir du bruit
        # ALTERNATIVES: conditional generation, progressive generation, cached generation
        # ENFANT: Le dessinateur transforme le bruit magique en fausses images
        
        d_output_fake = discriminator_cifar(fake_images.detach())
        # TECHNIQUE: Évaluation des fausses images avec .detach() pour éviter les gradients du générateur
        # ALTERNATIVES: torch.no_grad(), stop_gradient, separate forward passes
        # ENFANT: Le détective regarde les fausses images, mais on ne veut pas que ça influence le dessinateur
        
        errD_fake = criterion(d_output_fake, fake_labels)
        # TECHNIQUE: Calcul de la loss sur les fausses images
        # ALTERNATIVES: different loss weighting, adversarial losses, custom metrics
        # ENFANT: On regarde à quel point le détective s'est trompé sur les fausses images
        
        errD_fake.backward()
        # TECHNIQUE: Rétropropagation pour les fausses images
        # ALTERNATIVES: accumulated gradients, gradient penalty, spectral normalization
        # ENFANT: On calcule comment le détective doit changer pour mieux reconnaître les fausses images
        
        errD = errD_real + errD_fake
        # TECHNIQUE: Combinaison des deux losses du discriminateur
        # ALTERNATIVES: weighted combination, separate optimization, alternating updates
        # ENFANT: On additionne les deux erreurs du détective pour avoir son erreur totale
        
        optimizer_D_cifar.step()
        # TECHNIQUE: Mise à jour des paramètres du discriminateur
        # ALTERNATIVES: gradient clipping, learning rate scheduling, momentum updates
        # ENFANT: Le "professeur" applique les corrections au détective

        # Entraînement du Générateur
        optimizer_G_cifar.zero_grad()
        # TECHNIQUE: Reset des gradients pour le générateur
        # ALTERNATIVES: shared optimizer management, gradient accumulation strategies
        # ENFANT: On efface les anciennes leçons du dessinateur
        
        d_output_on_fake = discriminator_cifar(fake_images)
        # TECHNIQUE: Évaluation des images générées par le discriminateur (sans detach)
        # ALTERNATIVES: cached discriminator outputs, multiple discriminator calls
        # ENFANT: Le détective regarde les nouvelles images du dessinateur
        
        errG = criterion(d_output_on_fake, real_labels)
        # TECHNIQUE: Loss du générateur : il veut que ses fausses images soient classées comme vraies
        # ALTERNATIVES: feature matching, perceptual loss, WGAN loss, custom adversarial objectives
        # ENFANT: On regarde si le dessinateur arrive à tromper le détective (c'est le but !)
        
        errG.backward()
        # TECHNIQUE: Rétropropagation pour le générateur
        # ALTERNATIVES: gradient penalties, progressive growing, spectral normalization
        # ENFANT: On calcule comment le dessinateur doit changer pour mieux tromper le détective
        
        optimizer_G_cifar.step()
        # TECHNIQUE: Mise à jour des paramètres du générateur
        # ALTERNATIVES: different learning rates, momentum, adaptive methods
        # ENFANT: Le "professeur" applique les corrections au dessinateur

    print(f"[Epoch CIFAR-10 {epoch+1}/{epochs_finetune}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}")
    # TECHNIQUE: Logging des métriques avec formatage décimal
    # ALTERNATIVES: wandb.log(), tensorboard, csv logging, structured metrics
    # ENFANT: À la fin de chaque "année scolaire", on écrit les notes du détective et du dessinateur
    
    with torch.no_grad():
        fake_samples = generator_cifar(fixed_noise).detach().cpu()
    # TECHNIQUE: Génération d'échantillons sans calcul de gradients pour économiser la mémoire
    # ALTERNATIVES: eval() mode, inference mode, separate inference function
    # ENFANT: On demande au dessinateur de nous montrer ses progrès sans que ça compte pour ses notes
    
    save_image(fake_samples, f"cifar10_images/cifar10_fake_epoch_{epoch+1}.png", normalize=True)
    # TECHNIQUE: Sauvegarde d'images avec normalisation automatique
    # ALTERNATIVES: PIL.Image.save(), cv2.imwrite(), custom visualization, wandb.Image()
    # ENFANT: On sauvegarde les dessins du dessinateur dans un dossier pour les regarder plus tard

print("--- FIN DE LA PHASE 2 : Fine-tuning terminé. ---")
# TECHNIQUE: Message de fin d'entraînement
# ALTERNATIVES: logging with execution time, summary statistics, model checkpointing
# ENFANT: On annonce que l'école spécialisée est finie !

# --- 2.5. Génération et Visualisation finale ---
generator_cifar.eval()
# TECHNIQUE: Passage en mode évaluation (désactive dropout, batch norm en mode inference)
# ALTERNATIVES: torch.inference_mode(), context managers, functional evaluation
# ENFANT: On dit au dessinateur de passer en "mode examen" où il fait de son mieux

with torch.no_grad():
    # TECHNIQUE: Context manager pour désactiver le calcul de gradients
    # ALTERNATIVES: @torch.no_grad() decorator, torch.inference_mode(), manual gradient disabling
    # ENFANT: On dit à l'ordinateur de ne pas prendre de notes pendant que le dessinateur travaille
    
    final_noise = torch.randn(64, latent_dim, 1, 1, device=device)
    # TECHNIQUE: Génération de bruit pour 64 images finales
    # ALTERNATIVES: fixed seed noise, interpolated noise, conditional noise
    # ENFANT: On prépare 64 pots de "bruit magique" différents
    
    generated_images = generator_cifar(final_noise).detach().cpu()
    # TECHNIQUE: Génération finale avec transfert CPU et détachement du graphe
    # ALTERNATIVES: batch processing, streaming generation, memory-efficient generation
    # ENFANT: Le dessinateur transforme tous les pots de bruit en 64 belles images

plt.figure(figsize=(10, 10))
# TECHNIQUE: Création d'une grande figure pour la visualisation finale
# ALTERNATIVES: subplots, interactive plots, save without display
# ENFANT: On prépare une très grande feuille pour montrer tous les dessins

plt.axis("off")
# TECHNIQUE: Suppression des axes pour un affichage propre
# ALTERNATIVES: custom styling, seaborn styling, matplotlib themes
# ENFANT: On enlève les lignes et chiffres pour que ce soit plus joli

plt.title("Images CIFAR-10 finales générées par Transfer Learning")
# TECHNIQUE: Titre descriptif de la visualisation finale
# ALTERNATIVES: custom text positioning, multi-line titles, styled titles
# ENFANT: On écrit un beau titre qui explique ce qu'on regarde

plt.imshow(np.transpose(make_grid(generated_images, padding=2, normalize=True), (1, 2, 0)))
# TECHNIQUE: Affichage d'une grille d'images avec transposition des dimensions
# ALTERNATIVES: custom grid layouts, subplots, interactive galleries
# ENFANT: On arrange toutes les images en carré et on les montre sur notre grande feuille

plt.show()
# TECHNIQUE: Affichage final de la visualisation
# ALTERNATIVES: plt.savefig(), interactive display, web-based visualization
# ENFANT: On montre notre magnifique galerie d'art créée par notre IA !

In [ ]:
# Installation des dépendances pour Kaggle
!pip install pytorch-fid psutil

# Variables à définir dans votre notebook
latent_dim = 100
image_size = 64
lr_finetune = 0.0001
epochs_finetune = 50
batch_size = 64
betas = (0.5, 0.999)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.BCELoss()

In [ ]:
# =========================================================================
# PHASE 2 : TRANSFERT D'APPRENTISSAGE ET FINE-TUNING SUR CIFAR-10 (CIBLE)
# Version Enterprise avec Model Selection et Monitoring Industriel
# =========================================================================

import json
import time
from datetime import datetime
from pathlib import Path
import gc
import psutil
import numpy as np
from collections import defaultdict, deque
import warnings
warnings.filterwarnings('ignore')

# Configuration industrielle centralisée
class ModelConfig:
    """Configuration centralisée pour déploiement industriel"""
    def __init__(self):
        self.experiment_name = f"cifar10_transfer_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.model_registry_path = Path("model_registry")
        self.metrics_path = Path("metrics")
        self.artifacts_path = Path("artifacts")
        
        # Création des dossiers
        for path in [self.model_registry_path, self.metrics_path, self.artifacts_path]:
            path.mkdir(exist_ok=True)
        
        # Métriques de monitoring
        self.metrics_to_track = ['fid_score', 'inception_score', 'lpips_distance', 'model_size', 'inference_time']
        self.early_stopping_patience = 10
        self.model_selection_metric = 'fid_score'  # Plus bas = meilleur

config = ModelConfig()

# Classe de métriques industrielles
class GANMetrics:
    """Calcul de métriques industrielles pour évaluation GAN"""
    
    def __init__(self, device):
        self.device = device
        self.fid_calculator = None
        self.inception_model = None
        self._init_metrics_models()
    
    def _init_metrics_models(self):
        """Initialisation des modèles pour calcul de métriques"""
        try:
            # FID Calculator (Fréchet Inception Distance)
            from pytorch_fid import fid_score
            from torchvision.models import inception_v3
            
            self.inception_model = inception_v3(pretrained=True, transform_input=False).to(self.device)
            self.inception_model.eval()
            print("✓ Inception model loaded for FID/IS calculation")
            
        except ImportError:
            print("⚠ pytorch-fid not available. Install with: pip install pytorch-fid")
            print("⚠ Using alternative metrics calculation")
    
    def calculate_fid_score(self, real_images, fake_images):
        """Calcul du FID score (plus bas = meilleur)"""
        try:
            # Conversion en format approprié pour FID
            real_imgs = (real_images * 255).clamp(0, 255).byte()
            fake_imgs = (fake_images * 255).clamp(0, 255).byte()
            
            # Calcul simplifié du FID (approximation pour Kaggle)
            real_features = self._extract_inception_features(real_imgs)
            fake_features = self._extract_inception_features(fake_imgs)
            
            mu1, sigma1 = real_features.mean(0), np.cov(real_features.T)
            mu2, sigma2 = fake_features.mean(0), np.cov(fake_features.T)
            
            fid = self._calculate_frechet_distance(mu1, sigma1, mu2, sigma2)
            return float(fid)
            
        except Exception as e:
            print(f"⚠ FID calculation failed: {e}")
            return float('inf')
    
    def _extract_inception_features(self, images):
        """Extraction de features Inception"""
        with torch.no_grad():
            if len(images.shape) == 3:
                images = images.unsqueeze(0)
            
            # Resize to 299x299 for Inception
            images = torch.nn.functional.interpolate(images.float(), size=(299, 299), mode='bilinear')
            images = images.to(self.device)
            
            features = self.inception_model(images)
            return features.cpu().numpy()
    
    def _calculate_frechet_distance(self, mu1, sigma1, mu2, sigma2):
        """Calcul de la distance de Fréchet"""
        diff = mu1 - mu2
        covmean = np.sqrt(sigma1.dot(sigma2))
        
        if np.iscomplexobj(covmean):
            covmean = covmean.real
        
        return diff.dot(diff) + np.trace(sigma1 + sigma2 - 2*covmean)
    
    def calculate_inception_score(self, fake_images, splits=10):
        """Calcul de l'Inception Score (plus haut = meilleur)"""
        try:
            with torch.no_grad():
                fake_imgs = (fake_images * 255).clamp(0, 255).byte()
                
                # Resize pour Inception
                fake_imgs = torch.nn.functional.interpolate(fake_imgs.float(), size=(299, 299), mode='bilinear')
                fake_imgs = fake_imgs.to(self.device)
                
                probs = torch.nn.functional.softmax(self.inception_model(fake_imgs), dim=1)
                
                scores = []
                for i in range(splits):
                    part = probs[i * len(probs) // splits:(i + 1) * len(probs) // splits]
                    kl_div = part * (torch.log(part) - torch.log(part.mean(0, keepdim=True)))
                    kl_div = kl_div.sum(1).mean().exp()
                    scores.append(kl_div.item())
                
                return np.mean(scores), np.std(scores)
                
        except Exception as e:
            print(f"⚠ IS calculation failed: {e}")
            return 1.0, 0.0

# Classe de monitoring industriel
class ModelMonitor:
    """Monitoring industriel des performances et ressources"""
    
    def __init__(self, config):
        self.config = config
        self.metrics_history = defaultdict(list)
        self.resource_history = defaultdict(deque)
        self.best_models = {}
        self.early_stopping_counter = 0
        self.best_score = float('inf')
        
    def log_metrics(self, epoch, metrics_dict):
        """Logging des métriques avec timestamp"""
        timestamp = datetime.now().isoformat()
        
        log_entry = {
            'timestamp': timestamp,
            'epoch': epoch,
            **metrics_dict
        }
        
        # Sauvegarde des métriques
        for key, value in metrics_dict.items():
            self.metrics_history[key].append(value)
        
        # Sauvegarde JSON pour analyse
        metrics_file = self.config.metrics_path / f"{self.config.experiment_name}_metrics.jsonl"
        with open(metrics_file, 'a') as f:
            f.write(json.dumps(log_entry) + '\n')
        
        print(f"📊 Epoch {epoch}: {metrics_dict}")
    
    def monitor_resources(self):
        """Monitoring des ressources système"""
        cpu_percent = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.memory_allocated() / 1024**3  # GB
            gpu_cached = torch.cuda.memory_reserved() / 1024**3   # GB
        else:
            gpu_memory = gpu_cached = 0
        
        resources = {
            'cpu_percent': cpu_percent,
            'memory_used_gb': memory.used / 1024**3,
            'memory_available_gb': memory.available / 1024**3,
            'gpu_memory_gb': gpu_memory,
            'gpu_cached_gb': gpu_cached
        }
        
        # Garde seulement les 100 dernières mesures
        for key, value in resources.items():
            self.resource_history[key].append(value)
            if len(self.resource_history[key]) > 100:
                self.resource_history[key].popleft()
        
        return resources
    
    def check_early_stopping(self, current_score):
        """Early stopping basé sur la métrique principale"""
        if current_score < self.best_score:
            self.best_score = current_score
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return self.early_stopping_counter >= self.config.early_stopping_patience
    
    def save_model_checkpoint(self, generator, discriminator, epoch, metrics, is_best=False):
        """Sauvegarde de checkpoint avec métadonnées"""
        checkpoint_dir = self.config.model_registry_path / f"epoch_{epoch}"
        checkpoint_dir.mkdir(exist_ok=True)
        
        # Métadonnées du modèle
        metadata = {
            'epoch': epoch,
            'timestamp': datetime.now().isoformat(),
            'metrics': metrics,
            'model_size_mb': self._get_model_size(generator) + self._get_model_size(discriminator),
            'config': {
                'latent_dim': latent_dim,
                'image_channels': image_channels_cifar,
                'learning_rate': lr_finetune
            }
        }
        
        # Sauvegarde des modèles
        torch.save(generator.state_dict(), checkpoint_dir / "generator.pth")
        torch.save(discriminator.state_dict(), checkpoint_dir / "discriminator.pth")
        
        # Sauvegarde des métadonnées
        with open(checkpoint_dir / "metadata.json", 'w') as f:
            json.dump(metadata, f, indent=2)
        
        # Marquage du meilleur modèle
        if is_best:
            best_model_path = self.config.model_registry_path / "best_model"
            if best_model_path.exists():
                import shutil
                shutil.rmtree(best_model_path)
            shutil.copytree(checkpoint_dir, best_model_path)
            print(f"🏆 New best model saved (FID: {metrics.get('fid_score', 'N/A')})")
    
    def _get_model_size(self, model):
        """Calcul de la taille du modèle en MB"""
        param_size = sum(p.numel() * p.element_size() for p in model.parameters())
        buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
        return (param_size + buffer_size) / 1024**2

# Classe d'évaluation comparative
class ModelComparator:
    """Comparaison et sélection des meilleurs modèles"""
    
    def __init__(self, config):
        self.config = config
        self.model_scores = {}
        
    def evaluate_model_portfolio(self):
        """Évaluation de tous les modèles sauvegardés"""
        model_registry = self.config.model_registry_path
        evaluation_results = []
        
        for model_dir in model_registry.glob("epoch_*"):
            if not (model_dir / "metadata.json").exists():
                continue
                
            with open(model_dir / "metadata.json", 'r') as f:
                metadata = json.load(f)
            
            evaluation_results.append({
                'model_path': str(model_dir),
                'epoch': metadata['epoch'],
                'fid_score': metadata['metrics'].get('fid_score', float('inf')),
                'inception_score': metadata['metrics'].get('inception_score', 0),
                'model_size_mb': metadata['model_size_mb'],
                'inference_time': metadata['metrics'].get('inference_time', 0)
            })
        
        # Tri par métrique principale
        evaluation_results.sort(key=lambda x: x['fid_score'])
        
        # Sauvegarde du leaderboard
        leaderboard_path = self.config.artifacts_path / "model_leaderboard.json"
        with open(leaderboard_path, 'w') as f:
            json.dump(evaluation_results, f, indent=2)
        
        print("🏅 Model Leaderboard (Top 5):")
        for i, result in enumerate(evaluation_results[:5]):
            print(f"  {i+1}. Epoch {result['epoch']}: FID={result['fid_score']:.3f}, "
                  f"Size={result['model_size_mb']:.1f}MB")
        
        return evaluation_results

# Code principal amélioré
print("\n--- DÉBUT DE LA PHASE 2 : TRANSFERT INDUSTRIEL SUR CIFAR-10 ---")

# Initialisation des composants industriels
monitor = ModelMonitor(config)
metrics_calculator = GANMetrics(device)
comparator = ModelComparator(config)

# --- 2.1. Préparation des données CIFAR-10 (inchangé) ---
image_channels_cifar = 3
transform_cifar = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
dataloader_cifar = DataLoader(cifar_dataset, batch_size=batch_size, shuffle=True)

# Dataset de validation pour métriques
cifar_val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)
val_dataloader = DataLoader(cifar_val_dataset, batch_size=64, shuffle=False)

# Échantillon de vraies images pour calcul FID
real_samples_for_metrics = next(iter(val_dataloader))[0][:100]

print(f"🎯 Experiment: {config.experiment_name}")
print(f"📁 Artifacts will be saved to: {config.artifacts_path}")

# --- 2.2. Transfert des poids (code existant) ---
generator_cifar = Generator(latent_dim, image_channels_cifar).to(device)
discriminator_cifar = Discriminator(image_channels_cifar).to(device)

# ... (code de transfert existant) ...

# --- 2.3. Fine-Tuning avec monitoring industriel ---
params_g_finetune = filter(lambda p: p.requires_grad, generator_cifar.parameters())
params_d_finetune = filter(lambda p: p.requires_grad, discriminator_cifar.parameters())

optimizer_G_cifar = optim.Adam(params_g_finetune, lr=lr_finetune, betas=betas)
optimizer_D_cifar = optim.Adam(params_d_finetune, lr=lr_finetune, betas=betas)

# Learning rate scheduler industriel
scheduler_G = optim.lr_scheduler.ReduceLROnPlateau(optimizer_G_cifar, 'min', patience=5, factor=0.5)
scheduler_D = optim.lr_scheduler.ReduceLROnPlateau(optimizer_D_cifar, 'min', patience=5, factor=0.5)

print(f"\n🚀 Starting industrial fine-tuning...")
print(f"📊 Tracking metrics: {config.metrics_to_track}")

# Variables de tracking
training_start_time = time.time()
epoch_times = []

for epoch in range(epochs_finetune):
    epoch_start_time = time.time()
    
    # Métriques d'époque
    epoch_metrics = {
        'loss_d': 0.0,
        'loss_g': 0.0,
        'loss_d_real': 0.0,
        'loss_d_fake': 0.0
    }
    
    generator_cifar.train()
    discriminator_cifar.train()
    
    for i, (real_images, _) in enumerate(dataloader_cifar):
        real_images = real_images.to(device)
        batch_size_current = real_images.size(0)
        
        real_labels = torch.ones(batch_size_current, device=device)
        fake_labels = torch.zeros(batch_size_current, device=device)

        # --- Entraînement Discriminateur ---
        optimizer_D_cifar.zero_grad()
        
        d_output_real = discriminator_cifar(real_images)
        errD_real = criterion(d_output_real, real_labels)
        errD_real.backward()
        
        noise = torch.randn(batch_size_current, latent_dim, 1, 1, device=device)
        fake_images = generator_cifar(noise)
        d_output_fake = discriminator_cifar(fake_images.detach())
        errD_fake = criterion(d_output_fake, fake_labels)
        errD_fake.backward()
        
        errD = errD_real + errD_fake
        optimizer_D_cifar.step()

        # --- Entraînement Générateur ---
        optimizer_G_cifar.zero_grad()
        d_output_on_fake = discriminator_cifar(fake_images)
        errG = criterion(d_output_on_fake, real_labels)
        errG.backward()
        optimizer_G_cifar.step()
        
        # Accumulation des métriques
        epoch_metrics['loss_d'] += errD.item()
        epoch_metrics['loss_g'] += errG.item()
        epoch_metrics['loss_d_real'] += errD_real.item()
        epoch_metrics['loss_d_fake'] += errD_fake.item()
        
        # Monitoring des ressources (toutes les 50 itérations)
        if i % 50 == 0:
            resources = monitor.monitor_resources()
            
        # Nettoyage mémoire périodique
        if i % 100 == 0:
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            gc.collect()
    
    # Calcul des moyennes d'époque
    num_batches = len(dataloader_cifar)
    for key in epoch_metrics:
        epoch_metrics[key] /= num_batches
    
    epoch_time = time.time() - epoch_start_time
    epoch_times.append(epoch_time)
    
    # --- Évaluation industrielle ---
    if epoch % 2 == 0 or epoch == epochs_finetune - 1:  # Évaluation tous les 2 époques
        generator_cifar.eval()
        
        with torch.no_grad():
            # Génération d'échantillons pour métriques
            eval_noise = torch.randn(100, latent_dim, 1, 1, device=device)
            fake_samples = generator_cifar(eval_noise)
            
            # Calcul des métriques industrielles
            start_inference = time.time()
            fid_score = metrics_calculator.calculate_fid_score(
                real_samples_for_metrics.to(device), 
                fake_samples
            )
            inference_time = time.time() - start_inference
            
            inception_score, inception_std = metrics_calculator.calculate_inception_score(fake_samples)
            
            # Métriques complètes
            complete_metrics = {
                **epoch_metrics,
                'fid_score': fid_score,
                'inception_score': inception_score,
                'inception_std': inception_std,
                'inference_time': inference_time,
                'epoch_time': epoch_time,
                'lr_g': optimizer_G_cifar.param_groups[0]['lr'],
                'lr_d': optimizer_D_cifar.param_groups[0]['lr']
            }
            
            # Logging industriel
            monitor.log_metrics(epoch, complete_metrics)
            
            # Sauvegarde de checkpoint
            is_best = fid_score < monitor.best_score
            monitor.save_model_checkpoint(
                generator_cifar, discriminator_cifar, 
                epoch, complete_metrics, is_best
            )
            
            # Mise à jour des schedulers
            scheduler_G.step(fid_score)
            scheduler_D.step(fid_score)
            
            # Early stopping check
            if monitor.check_early_stopping(fid_score):
                print(f"🛑 Early stopping triggered at epoch {epoch}")
                break
            
            # Sauvegarde d'images avec métriques
            save_image(
                fake_samples[:64], 
                config.artifacts_path / f"epoch_{epoch}_fid_{fid_score:.3f}.png", 
                normalize=True, nrow=8
            )
    
    # Affichage des métriques
    print(f"[Epoch {epoch+1}/{epochs_finetune}] "
          f"Loss_D: {epoch_metrics['loss_d']:.4f} "
          f"Loss_G: {epoch_metrics['loss_g']:.4f} "
          f"Time: {epoch_time:.1f}s")

# --- Analyse finale et sélection du meilleur modèle ---
training_time = time.time() - training_start_time

print(f"\n🎯 Training completed in {training_time/3600:.2f} hours")
print(f"⚡ Average epoch time: {np.mean(epoch_times):.1f}s")

# Évaluation comparative finale
best_models = comparator.evaluate_model_portfolio()

# Génération du rapport final
final_report = {
    'experiment_name': config.experiment_name,
    'total_training_time_hours': training_time / 3600,
    'total_epochs_trained': epoch + 1,
    'best_model': best_models[0] if best_models else None,
    'avg_epoch_time_seconds': np.mean(epoch_times),
    'final_metrics': monitor.metrics_history,
    'early_stopped': monitor.early_stopping_counter >= config.early_stopping_patience
}

# Sauvegarde du rapport
report_path = config.artifacts_path / "final_report.json"
with open(report_path, 'w') as f:
    json.dump(final_report, f, indent=2, default=str)

print(f"📋 Final report saved to: {report_path}")
print(f"🏆 Best model FID score: {best_models[0]['fid_score']:.3f}")

# --- Code de test pour Kaggle ---
def kaggle_test_suite():
    """Suite de tests spécifiques pour environnement Kaggle"""
    print("\n🧪 Running Kaggle-specific tests...")
    
    # Test 1: Vérification des chemins et permissions
    test_paths = [config.model_registry_path, config.metrics_path, config.artifacts_path]
    for path in test_paths:
        assert path.exists(), f"Path {path} not accessible"
        # Test d'écriture
        test_file = path / "test_write.tmp"
        test_file.write_text("test")
        test_file.unlink()
    print("✅ File system access test passed")
    
    # Test 2: Vérification mémoire GPU/CPU
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"✅ GPU Memory available: {gpu_memory:.1f} GB")
        assert gpu_memory > 10, "Insufficient GPU memory for training"
    
    # Test 3: Génération rapide
    with torch.no_grad():
        test_noise = torch.randn(4, latent_dim, 1, 1, device=device)
        if 'generator_cifar' in locals():
            test_images = generator_cifar(test_noise)
            assert test_images.shape == (4, 3, image_size, image_size), "Wrong output shape"
    print("✅ Model generation test passed")
    
    # Test 4: Métriques calculation
    if len(monitor.metrics_history) > 0:
        latest_fid = monitor.metrics_history['fid_score'][-1]
        assert latest_fid < 300, f"FID score too high: {latest_fid}"
    print("✅ Metrics validation test passed")
    
    print("🎉 All Kaggle tests passed!")

# Exécution des tests Kaggle
try:
    kaggle_test_suite()
except Exception as e:
    print(f"⚠ Kaggle test failed: {e}")

print("\n--- FIN DE LA PHASE 2 INDUSTRIELLE ---")

In [ ]:
# =========================================================================
# PHASE 2 : TRANSFERT INDUSTRIEL CORRIGÉ - Métriques robustes pour Kaggle
# =========================================================================

import json
import time
from datetime import datetime
from pathlib import Path
import gc
import psutil
import numpy as np
from collections import defaultdict, deque
import warnings
warnings.filterwarnings('ignore')

# Installation des dépendances manquantes pour Kaggle
try:
    import torch
    import torchvision
    from torchvision import transforms, datasets
    from torch.utils.data import DataLoader
    import torch.nn as nn
    import torch.optim as optim
    from torchvision.utils import save_image, make_grid
    import matplotlib.pyplot as plt
    from scipy import linalg
    from sklearn.metrics import accuracy_score
except ImportError as e:
    print(f"Installing missing dependencies: {e}")

# Classe de métriques robustes pour Kaggle
class RobustGANMetrics:
    """Métriques GAN robustes adaptées à l'environnement Kaggle"""
    
    def __init__(self, device):
        self.device = device
        self.inception_model = None
        self.classifier_model = None
        self._init_lightweight_models()
    
    def _init_lightweight_models(self):
        """Initialisation de modèles légers pour métriques"""
        try:
            # Modèle léger pour classification (alternative à Inception)
            from torchvision.models import resnet18
            self.classifier_model = resnet18(pretrained=True)
            self.classifier_model.eval()
            self.classifier_model = self.classifier_model.to(self.device)
            print("✓ ResNet18 loaded for lightweight metrics")
            
        except Exception as e:
            print(f"⚠ Could not load pretrained models: {e}")
            print("📊 Using basic statistical metrics instead")
    
    def calculate_lightweight_fid(self, real_images, fake_images):
        """Calcul FID simplifié et robuste"""
        try:
            # Vérification des inputs
            if real_images.numel() == 0 or fake_images.numel() == 0:
                return float('inf')
            
            # Normalisation des images
            real_imgs = self._normalize_images(real_images)
            fake_imgs = self._normalize_images(fake_images)
            
            # Si pas de modèle préentraîné, utiliser des statistiques simples
            if self.classifier_model is None:
                return self._calculate_statistical_distance(real_imgs, fake_imgs)
            
            # Extraction de features avec modèle léger
            real_features = self._extract_resnet_features(real_imgs)
            fake_features = self._extract_resnet_features(fake_imgs)
            
            # Calcul FID robuste avec gestion des cas limites
            fid_score = self._compute_robust_fid(real_features, fake_features)
            
            # Validation du résultat
            if np.isnan(fid_score) or np.isinf(fid_score):
                print("⚠ FID calculation resulted in nan/inf, using fallback metric")
                return self._calculate_statistical_distance(real_imgs, fake_imgs)
                
            return float(fid_score)
            
        except Exception as e:
            print(f"⚠ FID calculation error: {e}")
            return self._calculate_statistical_distance(real_images, fake_images)
    
    def _normalize_images(self, images):
        """Normalisation robuste des images"""
        if images.dim() == 3:
            images = images.unsqueeze(0)
        
        # Clamp et normalisation
        images = torch.clamp(images, -1, 1)
        images = (images + 1) / 2  # [-1,1] -> [0,1]
        
        return images
    
    def _extract_resnet_features(self, images):
        """Extraction de features avec ResNet18"""
        with torch.no_grad():
            # Resize pour ResNet
            if images.shape[-1] != 224:
                images = torch.nn.functional.interpolate(
                    images, size=(224, 224), mode='bilinear', align_corners=False
                )
            
            # Forward pass jusqu'aux features
            x = self.classifier_model.conv1(images)
            x = self.classifier_model.bn1(x)
            x = self.classifier_model.relu(x)
            x = self.classifier_model.maxpool(x)
            
            x = self.classifier_model.layer1(x)
            x = self.classifier_model.layer2(x)
            x = self.classifier_model.layer3(x)
            x = self.classifier_model.layer4(x)
            
            x = self.classifier_model.avgpool(x)
            features = torch.flatten(x, 1)
            
            return features.cpu().numpy()
    
    def _compute_robust_fid(self, real_features, fake_features):
        """Calcul FID robuste avec gestion d'erreurs"""
        try:
            # Calcul des statistiques
            mu1 = np.mean(real_features, axis=0)
            mu2 = np.mean(fake_features, axis=0)
            
            sigma1 = np.cov(real_features, rowvar=False)
            sigma2 = np.cov(fake_features, rowvar=False)
            
            # Gestion des matrices singulières
            if sigma1.ndim == 0:
                sigma1 = np.array([[sigma1]])
            if sigma2.ndim == 0:
                sigma2 = np.array([[sigma2]])
            
            # Ajout de régularisation pour éviter les matrices singulières
            eps = 1e-6
            sigma1 += eps * np.eye(sigma1.shape[0])
            sigma2 += eps * np.eye(sigma2.shape[0])
            
            # Calcul de la distance de Fréchet
            diff = mu1 - mu2
            
            # Calcul de la racine carrée de la matrice
            covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
            
            # Gestion des nombres complexes
            if np.iscomplexobj(covmean):
                if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
                    print("⚠ Warning: Imaginary component in covariance mean")
                covmean = covmean.real
            
            # Calcul final du FID
            fid = (diff.dot(diff) + np.trace(sigma1) + 
                   np.trace(sigma2) - 2 * np.trace(covmean))
            
            return fid
            
        except Exception as e:
            print(f"⚠ Robust FID calculation failed: {e}")
            return float('inf')
    
    def _calculate_statistical_distance(self, real_images, fake_images):
        """Métrique de fallback basée sur les statistiques d'images"""
        try:
            # Conversion en numpy
            real_np = real_images.detach().cpu().numpy()
            fake_np = fake_images.detach().cpu().numpy()
            
            # Statistiques de base
            real_mean = np.mean(real_np)
            fake_mean = np.mean(fake_np)
            real_std = np.std(real_np)
            fake_std = np.std(fake_np)
            
            # Distance statistique simple
            mean_diff = abs(real_mean - fake_mean)
            std_diff = abs(real_std - fake_std)
            
            # Score combiné (plus bas = meilleur)
            stat_distance = mean_diff * 100 + std_diff * 50
            
            return float(stat_distance)
            
        except Exception as e:
            print(f"⚠ Statistical distance calculation failed: {e}")
            return 100.0  # Score par défaut
    
    def calculate_diversity_score(self, fake_images):
        """Score de diversité des images générées"""
        try:
            if fake_images.numel() == 0:
                return 0.0
            
            # Calcul de la diversité basée sur la variance des pixels
            fake_flat = fake_images.view(fake_images.size(0), -1)
            
            # Variance moyenne entre images
            pairwise_distances = torch.cdist(fake_flat, fake_flat)
            
            # Exclusion de la diagonale (distance à soi-même)
            mask = ~torch.eye(pairwise_distances.size(0), dtype=torch.bool)
            avg_distance = pairwise_distances[mask].mean()
            
            return float(avg_distance)
            
        except Exception as e:
            print(f"⚠ Diversity score calculation failed: {e}")
            return 0.0
    
    def calculate_quality_score(self, images):
        """Score de qualité basé sur la netteté et le contraste"""
        try:
            if images.numel() == 0:
                return 0.0
            
            # Conversion en format approprié
            imgs = torch.clamp((images + 1) / 2, 0, 1)
            
            # Calcul du gradient (netteté)
            grad_x = torch.abs(imgs[:, :, :, 1:] - imgs[:, :, :, :-1])
            grad_y = torch.abs(imgs[:, :, 1:, :] - imgs[:, :, :-1, :])
            
            sharpness = (grad_x.mean() + grad_y.mean()) / 2
            
            # Calcul du contraste
            contrast = torch.std(imgs)
            
            # Score combiné
            quality = (sharpness * 10 + contrast * 5).item()
            
            return quality
            
        except Exception as e:
            print(f"⚠ Quality score calculation failed: {e}")
            return 0.0

# Classe de monitoring améliorée
class ImprovedModelMonitor:
    """Monitoring amélioré avec métriques robustes"""
    
    def __init__(self, config):
        self.config = config
        self.metrics_history = defaultdict(list)
        self.resource_history = defaultdict(deque)
        self.best_models = {}
        self.early_stopping_counter = 0
        self.best_score = float('inf')
        self.metrics_calculator = RobustGANMetrics(device)
        
        # Métriques alternatives pour Kaggle
        self.alternative_metrics = [
            'statistical_distance', 'diversity_score', 'quality_score', 
            'loss_stability', 'gradient_norm'
        ]
    
    def comprehensive_evaluation(self, generator, real_samples, epoch):
        """Évaluation complète avec métriques multiples"""
        try:
            generator.eval()
            metrics = {}
            
            with torch.no_grad():
                # Génération d'échantillons
                eval_noise = torch.randn(min(100, len(real_samples)), latent_dim, 1, 1, device=device)
                fake_samples = generator(eval_noise)
                
                # Limitation du nombre d'échantillons pour éviter les problèmes mémoire
                n_samples = min(50, len(real_samples), len(fake_samples))
                real_subset = real_samples[:n_samples]
                fake_subset = fake_samples[:n_samples]
                
                # Métriques principales
                start_time = time.time()
                
                # FID robuste
                fid_score = self.metrics_calculator.calculate_lightweight_fid(
                    real_subset, fake_subset
                )
                metrics['fid_score'] = fid_score
                
                # Métriques alternatives
                diversity = self.metrics_calculator.calculate_diversity_score(fake_subset)
                quality = self.metrics_calculator.calculate_quality_score(fake_subset)
                
                metrics.update({
                    'diversity_score': diversity,
                    'quality_score': quality,
                    'inference_time': time.time() - start_time,
                    'n_samples_evaluated': n_samples
                })
                
                # Score composite pour sélection de modèle
                if not np.isnan(fid_score) and not np.isinf(fid_score):
                    composite_score = fid_score
                else:
                    # Fallback: utiliser le score de qualité inversé
                    composite_score = max(0, 100 - quality * 10)
                
                metrics['composite_score'] = composite_score
                
                print(f"📊 Epoch {epoch} Metrics:")
                print(f"   FID: {fid_score:.3f}")
                print(f"   Diversity: {diversity:.3f}")
                print(f"   Quality: {quality:.3f}")
                print(f"   Composite: {composite_score:.3f}")
                
                return metrics
                
        except Exception as e:
            print(f"⚠ Comprehensive evaluation failed: {e}")
            # Métriques par défaut en cas d'erreur
            return {
                'fid_score': 100.0,
                'diversity_score': 0.0,
                'quality_score': 0.0,
                'composite_score': 100.0,
                'inference_time': 0.0,
                'evaluation_error': str(e)
            }
    
    def check_early_stopping(self, current_score):
        """Early stopping basé sur le score composite"""
        if np.isnan(current_score) or np.isinf(current_score):
            current_score = float('inf')
            
        if current_score < self.best_score:
            self.best_score = current_score
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return self.early_stopping_counter >= self.config.early_stopping_patience
    
    def log_metrics(self, epoch, metrics_dict):
        """Logging robuste des métriques"""
        timestamp = datetime.now().isoformat()
        
        # Nettoyage des valeurs NaN/Inf
        clean_metrics = {}
        for key, value in metrics_dict.items():
            if isinstance(value, (int, float)):
                if np.isnan(value) or np.isinf(value):
                    clean_metrics[key] = None
                else:
                    clean_metrics[key] = float(value)
            else:
                clean_metrics[key] = value
        
        log_entry = {
            'timestamp': timestamp,
            'epoch': epoch,
            **clean_metrics
        }
        
        # Sauvegarde des métriques
        for key, value in clean_metrics.items():
            if value is not None:
                self.metrics_history[key].append(value)
        
        # Sauvegarde JSON
        metrics_file = self.config.metrics_path / f"{self.config.experiment_name}_metrics.jsonl"
        with open(metrics_file, 'a') as f:
            f.write(json.dumps(log_entry, default=str) + '\n')
        
        print(f"📊 Epoch {epoch}: {clean_metrics}")
    
    def save_model_checkpoint(self, generator, discriminator, epoch, metrics, is_best=False):
        """Sauvegarde robuste avec validation"""
        try:
            checkpoint_dir = self.config.model_registry_path / f"epoch_{epoch}"
            checkpoint_dir.mkdir(exist_ok=True)
            
            # Validation des modèles avant sauvegarde
            if not hasattr(generator, 'state_dict') or not hasattr(discriminator, 'state_dict'):
                raise ValueError("Invalid model objects for saving")
            
            # Métadonnées avec validation
            clean_metrics = {k: v for k, v in metrics.items() 
                           if not (isinstance(v, float) and (np.isnan(v) or np.isinf(v)))}
            
            metadata = {
                'epoch': epoch,
                'timestamp': datetime.now().isoformat(),
                'metrics': clean_metrics,
                'model_size_mb': self._get_model_size(generator) + self._get_model_size(discriminator),
                'is_best': is_best,
                'config': {
                    'latent_dim': latent_dim,
                    'image_channels': image_channels_cifar,
                    'learning_rate': lr_finetune
                }
            }
            
            # Sauvegarde des modèles
            torch.save(generator.state_dict(), checkpoint_dir / "generator.pth")
            torch.save(discriminator.state_dict(), checkpoint_dir / "discriminator.pth")
            
            # Sauvegarde des métadonnées
            with open(checkpoint_dir / "metadata.json", 'w') as f:
                json.dump(metadata, f, indent=2, default=str)
            
            # Marquage du meilleur modèle
            if is_best:
                best_model_path = self.config.model_registry_path / "best_model"
                if best_model_path.exists():
                    import shutil
                    shutil.rmtree(best_model_path)
                
                import shutil
                shutil.copytree(checkpoint_dir, best_model_path)
                
                composite_score = clean_metrics.get('composite_score', 'N/A')
                print(f"🏆 New best model saved (Composite Score: {composite_score})")
                
        except Exception as e:
            print(f"⚠ Model checkpoint saving failed: {e}")
    
    def _get_model_size(self, model):
        """Calcul robuste de la taille du modèle"""
        try:
            param_size = sum(p.numel() * p.element_size() for p in model.parameters())
            buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
            return (param_size + buffer_size) / 1024**2
        except Exception as e:
            print(f"⚠ Model size calculation failed: {e}")
            return 0.0

# Configuration mise à jour
class ImprovedModelConfig:
    def __init__(self):
        self.experiment_name = f"cifar10_robust_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.model_registry_path = Path("model_registry")
        self.metrics_path = Path("metrics")
        self.artifacts_path = Path("artifacts")
        
        for path in [self.model_registry_path, self.metrics_path, self.artifacts_path]:
            path.mkdir(exist_ok=True)
        
        self.early_stopping_patience = 8  # Plus conservateur
        self.model_selection_metric = 'composite_score'
        self.evaluation_frequency = 3  # Évaluation tous les 3 epochs

# Tests Kaggle améliorés
def improved_kaggle_test_suite():
    """Suite de tests robuste pour Kaggle"""
    print("\n🧪 Running improved Kaggle tests...")
    
    try:
        # Test 1: Vérification système
        print(f"🖥 Python version: {sys.version.split()[0]}")
        print(f"🔥 PyTorch version: {torch.__version__}")
        print(f"🖼 Torchvision version: {torchvision.__version__}")
        
        # Test 2: Mémoire disponible
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
            print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
        
        # Test 3: Test des métriques
        test_metrics = RobustGANMetrics(device)
        test_real = torch.randn(4, 3, 64, 64).to(device)
        test_fake = torch.randn(4, 3, 64, 64).to(device)
        
        fid_test = test_metrics.calculate_lightweight_fid(test_real, test_fake)
        diversity_test = test_metrics.calculate_diversity_score(test_fake)
        quality_test = test_metrics.calculate_quality_score(test_fake)
        
        print(f"🧮 Metrics test - FID: {fid_test:.3f}, Diversity: {diversity_test:.3f}, Quality: {quality_test:.3f}")
        
        # Validation des métriques
        assert not np.isnan(fid_test), f"FID calculation returned NaN"
        assert not np.isnan(diversity_test), f"Diversity calculation returned NaN"
        assert not np.isnan(quality_test), f"Quality calculation returned NaN"
        
        print("✅ All improved Kaggle tests passed!")
        return True
        
    except Exception as e:
        print(f"❌ Kaggle test failed: {e}")
        return False

# Application du code corrigé
print("\n--- DÉBUT DE LA PHASE 2 INDUSTRIELLE CORRIGÉE ---")

# Initialisation des composants améliorés
config = ImprovedModelConfig()
monitor = ImprovedModelMonitor(config)

print(f"🎯 Experiment: {config.experiment_name}")
print(f"🔧 Using robust metrics with fallback strategies")

# Test initial
test_passed = improved_kaggle_test_suite()
if not test_passed:
    print("⚠ Some tests failed, but continuing with robust fallbacks...")

# Le reste du code d'entraînement reste similaire mais avec monitor amélioré
# Remplacer l'appel d'évaluation par:
"""
# Dans la boucle d'entraînement, remplacer la section d'évaluation par:
if epoch % config.evaluation_frequency == 0 or epoch == epochs_finetune - 1:
    # Évaluation complète avec métriques robustes
    evaluation_metrics = monitor.comprehensive_evaluation(
        generator_cifar, real_samples_for_metrics, epoch
    )
    
    # Métriques d'entraînement
    complete_metrics = {
        **epoch_metrics,
        **evaluation_metrics,
        'epoch_time': epoch_time,
        'lr_g': optimizer_G_cifar.param_groups[0]['lr'],
        'lr_d': optimizer_D_cifar.param_groups[0]['lr']
    }
    
    # Logging et sauvegarde
    monitor.log_metrics(epoch, complete_metrics)
    
    # Sélection basée sur le score composite
    composite_score = evaluation_metrics.get('composite_score', float('inf'))
    is_best = composite_score < monitor.best_score
    
    monitor.save_model_checkpoint(
        generator_cifar, discriminator_cifar, 
        epoch, complete_metrics, is_best
    )
    
    # Early stopping
    if monitor.check_early_stopping(composite_score):
        print(f"🛑 Early stopping at epoch {epoch}")
        break
"""

# Code principal amélioré
# print("\n--- DÉBUT DE LA PHASE 2 : TRANSFERT INDUSTRIEL SUR CIFAR-10 ---")

# Initialisation des composants industriels
monitor = ModelMonitor(config)
metrics_calculator = GANMetrics(device)
comparator = ModelComparator(config)

# --- 2.1. Préparation des données CIFAR-10 (inchangé) ---
image_channels_cifar = 3
transform_cifar = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
dataloader_cifar = DataLoader(cifar_dataset, batch_size=batch_size, shuffle=True)

# Dataset de validation pour métriques
cifar_val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)
val_dataloader = DataLoader(cifar_val_dataset, batch_size=64, shuffle=False)

# Échantillon de vraies images pour calcul FID
real_samples_for_metrics = next(iter(val_dataloader))[0][:100]

print(f"🎯 Experiment: {config.experiment_name}")
print(f"📁 Artifacts will be saved to: {config.artifacts_path}")

# --- 2.2. Transfert des poids (code existant) ---
generator_cifar = Generator(latent_dim, image_channels_cifar).to(device)
discriminator_cifar = Discriminator(image_channels_cifar).to(device)

# ... (code de transfert existant) ...

# --- 2.3. Fine-Tuning avec monitoring industriel ---
params_g_finetune = filter(lambda p: p.requires_grad, generator_cifar.parameters())
params_d_finetune = filter(lambda p: p.requires_grad, discriminator_cifar.parameters())

optimizer_G_cifar = optim.Adam(params_g_finetune, lr=lr_finetune, betas=betas)
optimizer_D_cifar = optim.Adam(params_d_finetune, lr=lr_finetune, betas=betas)

# Learning rate scheduler industriel
scheduler_G = optim.lr_scheduler.ReduceLROnPlateau(optimizer_G_cifar, 'min', patience=5, factor=0.5)
scheduler_D = optim.lr_scheduler.ReduceLROnPlateau(optimizer_D_cifar, 'min', patience=5, factor=0.5)

print(f"\n🚀 Starting industrial fine-tuning...")
print(f"📊 Tracking metrics: {config.metrics_to_track}")

# Variables de tracking
training_start_time = time.time()
epoch_times = []

for epoch in range(epochs_finetune):
    epoch_start_time = time.time()
    
    # Métriques d'époque
    epoch_metrics = {
        'loss_d': 0.0,
        'loss_g': 0.0,
        'loss_d_real': 0.0,
        'loss_d_fake': 0.0
    }
    
    generator_cifar.train()
    discriminator_cifar.train()
    
    for i, (real_images, _) in enumerate(dataloader_cifar):
        real_images = real_images.to(device)
        batch_size_current = real_images.size(0)
        
        real_labels = torch.ones(batch_size_current, device=device)
        fake_labels = torch.zeros(batch_size_current, device=device)

        # --- Entraînement Discriminateur ---
        optimizer_D_cifar.zero_grad()
        
        d_output_real = discriminator_cifar(real_images)
        errD_real = criterion(d_output_real, real_labels)
        errD_real.backward()
        
        noise = torch.randn(batch_size_current, latent_dim, 1, 1, device=device)
        fake_images = generator_cifar(noise)
        d_output_fake = discriminator_cifar(fake_images.detach())
        errD_fake = criterion(d_output_fake, fake_labels)
        errD_fake.backward()
        
        errD = errD_real + errD_fake
        optimizer_D_cifar.step()

        # --- Entraînement Générateur ---
        optimizer_G_cifar.zero_grad()
        d_output_on_fake = discriminator_cifar(fake_images)
        errG = criterion(d_output_on_fake, real_labels)
        errG.backward()
        optimizer_G_cifar.step()
        
        # Accumulation des métriques
        epoch_metrics['loss_d'] += errD.item()
        epoch_metrics['loss_g'] += errG.item()
        epoch_metrics['loss_d_real'] += errD_real.item()
        epoch_metrics['loss_d_fake'] += errD_fake.item()
        
        # Monitoring des ressources (toutes les 50 itérations)
        if i % 50 == 0:
            resources = monitor.monitor_resources()
            
        # Nettoyage mémoire périodique
        if i % 100 == 0:
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            gc.collect()
    
    # Calcul des moyennes d'époque
    num_batches = len(dataloader_cifar)
    for key in epoch_metrics:
        epoch_metrics[key] /= num_batches
    
    epoch_time = time.time() - epoch_start_time
    epoch_times.append(epoch_time)
    # Dans la boucle d'entraînement, remplacer la section d'évaluation par:
    if epoch % config.evaluation_frequency == 0 or epoch == epochs_finetune - 1:
        # Évaluation complète avec métriques robustes
        evaluation_metrics = monitor.comprehensive_evaluation(
            generator_cifar, real_samples_for_metrics, epoch
        )
        
        # Métriques d'entraînement
        complete_metrics = {
            **epoch_metrics,
            **evaluation_metrics,
            'epoch_time': epoch_time,
            'lr_g': optimizer_G_cifar.param_groups[0]['lr'],
            'lr_d': optimizer_D_cifar.param_groups[0]['lr']
        }
        
        # Logging et sauvegarde
        monitor.log_metrics(epoch, complete_metrics)
        
        # Sélection basée sur le score composite
        composite_score = evaluation_metrics.get('composite_score', float('inf'))
        is_best = composite_score < monitor.best_score
        
        monitor.save_model_checkpoint(
            generator_cifar, discriminator_cifar, 
            epoch, complete_metrics, is_best
        )
        
        # Early stopping
        if monitor.check_early_stopping(composite_score):
            print(f"🛑 Early stopping at epoch {epoch}")
            break

    # # --- Évaluation industrielle ---
    # if epoch % 2 == 0 or epoch == epochs_finetune - 1:  # Évaluation tous les 2 époques
    #     generator_cifar.eval()
        
    #     with torch.no_grad():
    #         # Génération d'échantillons pour métriques
    #         eval_noise = torch.randn(100, latent_dim, 1, 1, device=device)
    #         fake_samples = generator_cifar(eval_noise)
            
    #         # Calcul des métriques industrielles
    #         start_inference = time.time()
    #         fid_score = metrics_calculator.calculate_fid_score(
    #             real_samples_for_metrics.to(device), 
    #             fake_samples
    #         )
    #         inference_time = time.time() - start_inference
            
    #         inception_score, inception_std = metrics_calculator.calculate_inception_score(fake_samples)
            
    #         # Métriques complètes
    #         complete_metrics = {
    #             **epoch_metrics,
    #             'fid_score': fid_score,
    #             'inception_score': inception_score,
    #             'inception_std': inception_std,
    #             'inference_time': inference_time,
    #             'epoch_time': epoch_time,
    #             'lr_g': optimizer_G_cifar.param_groups[0]['lr'],
    #             'lr_d': optimizer_D_cifar.param_groups[0]['lr']
    #         }
            
    #         # Logging industriel
    #         monitor.log_metrics(epoch, complete_metrics)
            
    #         # Sauvegarde de checkpoint
    #         is_best = fid_score < monitor.best_score
    #         monitor.save_model_checkpoint(
    #             generator_cifar, discriminator_cifar, 
    #             epoch, complete_metrics, is_best
    #         )
            
    #         # Mise à jour des schedulers
    #         scheduler_G.step(fid_score)
    #         scheduler_D.step(fid_score)
            
    #         # Early stopping check
    #         if monitor.check_early_stopping(fid_score):
    #             print(f"🛑 Early stopping triggered at epoch {epoch}")
    #             break
            
            # Sauvegarde d'images avec métriques
            save_image(
                fake_samples[:64], 
                config.artifacts_path / f"epoch_{epoch}_fid_{fid_score:.3f}.png", 
                normalize=True, nrow=8
            )
    
    # Affichage des métriques
    print(f"[Epoch {epoch+1}/{epochs_finetune}] "
          f"Loss_D: {epoch_metrics['loss_d']:.4f} "
          f"Loss_G: {epoch_metrics['loss_g']:.4f} "
          f"Time: {epoch_time:.1f}s")

# --- Analyse finale et sélection du meilleur modèle ---
training_time = time.time() - training_start_time

print(f"\n🎯 Training completed in {training_time/3600:.2f} hours")
print(f"⚡ Average epoch time: {np.mean(epoch_times):.1f}s")

# Évaluation comparative finale
best_models = comparator.evaluate_model_portfolio()

# Génération du rapport final
final_report = {
    'experiment_name': config.experiment_name,
    'total_training_time_hours': training_time / 3600,
    'total_epochs_trained': epoch + 1,
    'best_model': best_models[0] if best_models else None,
    'avg_epoch_time_seconds': np.mean(epoch_times),
    'final_metrics': monitor.metrics_history,
    'early_stopped': monitor.early_stopping_counter >= config.early_stopping_patience
}

# Sauvegarde du rapport
report_path = config.artifacts_path / "final_report.json"
with open(report_path, 'w') as f:
    json.dump(final_report, f, indent=2, default=str)

print(f"📋 Final report saved to: {report_path}")
print(f"🏆 Best model FID score: {best_models[0]['fid_score']:.3f}")

# --- Code de test pour Kaggle ---
def kaggle_test_suite():
    """Suite de tests spécifiques pour environnement Kaggle"""
    print("\n🧪 Running Kaggle-specific tests...")
    
    # Test 1: Vérification des chemins et permissions
    test_paths = [config.model_registry_path, config.metrics_path, config.artifacts_path]
    for path in test_paths:
        assert path.exists(), f"Path {path} not accessible"
        # Test d'écriture
        test_file = path / "test_write.tmp"
        test_file.write_text("test")
        test_file.unlink()
    print("✅ File system access test passed")
    
    # Test 2: Vérification mémoire GPU/CPU
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"✅ GPU Memory available: {gpu_memory:.1f} GB")
        assert gpu_memory > 10, "Insufficient GPU memory for training"
    
    # Test 3: Génération rapide
    with torch.no_grad():
        test_noise = torch.randn(4, latent_dim, 1, 1, device=device)
        if 'generator_cifar' in locals():
            test_images = generator_cifar(test_noise)
            assert test_images.shape == (4, 3, image_size, image_size), "Wrong output shape"
    print("✅ Model generation test passed")
    
    # Test 4: Métriques calculation
    if len(monitor.metrics_history) > 0:
        latest_fid = monitor.metrics_history['fid_score'][-1]
        assert latest_fid < 300, f"FID score too high: {latest_fid}"
    print("✅ Metrics validation test passed")
    
    print("🎉 All Kaggle tests passed!")

# Exécution des tests Kaggle
try:
    kaggle_test_suite()
except Exception as e:
    print(f"⚠ Kaggle test failed: {e}")

print("\n--- FIN DE LA PHASE 2 INDUSTRIELLE ---")

In [ ]:
# =========================================================================
# PHASE 2 : TRANSFERT INDUSTRIEL CORRIGÉ - Métriques robustes pour Kaggle
# =========================================================================

import json
import time
import sys
from datetime import datetime
from pathlib import Path
import gc
import psutil
import numpy as np
from collections import defaultdict, deque
import warnings
warnings.filterwarnings('ignore')

# Installation des dépendances manquantes pour Kaggle
try:
    import torch
    import torchvision
    from torchvision import transforms, datasets
    from torch.utils.data import DataLoader
    import torch.nn as nn
    import torch.optim as optim
    from torchvision.utils import save_image, make_grid
    import matplotlib.pyplot as plt
    from scipy import linalg
    from sklearn.metrics import accuracy_score
except ImportError as e:
    print(f"Installing missing dependencies: {e}")

# Configuration des paramètres de base
image_size = 64
batch_size = 128
latent_dim = 100
lr_finetune = 0.0002
betas = (0.5, 0.999)
epochs_finetune = 100

# Initialisation du device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Classe de métriques robustes pour Kaggle
class RobustGANMetrics:
    """Métriques GAN robustes adaptées à l'environnement Kaggle"""
    
    def __init__(self, device):
        self.device = device
        self.inception_model = None
        self.classifier_model = None
        self._init_lightweight_models()
    
    def _init_lightweight_models(self):
        """Initialisation de modèles légers pour métriques"""
        try:
            # Modèle léger pour classification (alternative à Inception)
            from torchvision.models import resnet18
            self.classifier_model = resnet18(pretrained=True)
            self.classifier_model.eval()
            self.classifier_model = self.classifier_model.to(self.device)
            print("✓ ResNet18 loaded for lightweight metrics")
            
        except Exception as e:
            print(f"⚠ Could not load pretrained models: {e}")
            print("📊 Using basic statistical metrics instead")
    
    def calculate_lightweight_fid(self, real_images, fake_images):
        """Calcul FID simplifié et robuste"""
        try:
            # Vérification des inputs
            if real_images.numel() == 0 or fake_images.numel() == 0:
                return float('inf')
            
            # Normalisation des images
            real_imgs = self._normalize_images(real_images)
            fake_imgs = self._normalize_images(fake_images)
            
            # Si pas de modèle préentraîné, utiliser des statistiques simples
            if self.classifier_model is None:
                return self._calculate_statistical_distance(real_imgs, fake_imgs)
            
            # Extraction de features avec modèle léger
            real_features = self._extract_resnet_features(real_imgs)
            fake_features = self._extract_resnet_features(fake_imgs)
            
            # Calcul FID robuste avec gestion des cas limites
            fid_score = self._compute_robust_fid(real_features, fake_features)
            
            # Validation du résultat
            if np.isnan(fid_score) or np.isinf(fid_score):
                print("⚠ FID calculation resulted in nan/inf, using fallback metric")
                return self._calculate_statistical_distance(real_imgs, fake_imgs)
                
            return float(fid_score)
            
        except Exception as e:
            print(f"⚠ FID calculation error: {e}")
            return self._calculate_statistical_distance(real_images, fake_images)
    
    def _normalize_images(self, images):
        """Normalisation robuste des images"""
        if images.dim() == 3:
            images = images.unsqueeze(0)
        
        # Clamp et normalisation
        images = torch.clamp(images, -1, 1)
        images = (images + 1) / 2  # [-1,1] -> [0,1]
        
        return images
    
    def _extract_resnet_features(self, images):
        """Extraction de features avec ResNet18"""
        with torch.no_grad():
            # Resize pour ResNet
            if images.shape[-1] != 224:
                images = torch.nn.functional.interpolate(
                    images, size=(224, 224), mode='bilinear', align_corners=False
                )
            
            # Forward pass jusqu'aux features
            x = self.classifier_model.conv1(images)
            x = self.classifier_model.bn1(x)
            x = self.classifier_model.relu(x)
            x = self.classifier_model.maxpool(x)
            
            x = self.classifier_model.layer1(x)
            x = self.classifier_model.layer2(x)
            x = self.classifier_model.layer3(x)
            x = self.classifier_model.layer4(x)
            
            x = self.classifier_model.avgpool(x)
            features = torch.flatten(x, 1)
            
            return features.cpu().numpy()
    
    def _compute_robust_fid(self, real_features, fake_features):
        """Calcul FID robuste avec gestion d'erreurs"""
        try:
            # Calcul des statistiques
            mu1 = np.mean(real_features, axis=0)
            mu2 = np.mean(fake_features, axis=0)
            
            sigma1 = np.cov(real_features, rowvar=False)
            sigma2 = np.cov(fake_features, rowvar=False)
            
            # Gestion des matrices singulières
            if sigma1.ndim == 0:
                sigma1 = np.array([[sigma1]])
            if sigma2.ndim == 0:
                sigma2 = np.array([[sigma2]])
            
            # Ajout de régularisation pour éviter les matrices singulières
            eps = 1e-6
            sigma1 += eps * np.eye(sigma1.shape[0])
            sigma2 += eps * np.eye(sigma2.shape[0])
            
            # Calcul de la distance de Fréchet
            diff = mu1 - mu2
            
            # Calcul de la racine carrée de la matrice
            covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
            
            # Gestion des nombres complexes
            if np.iscomplexobj(covmean):
                if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
                    print("⚠ Warning: Imaginary component in covariance mean")
                covmean = covmean.real
            
            # Calcul final du FID
            fid = (diff.dot(diff) + np.trace(sigma1) + 
                   np.trace(sigma2) - 2 * np.trace(covmean))
            
            return fid
            
        except Exception as e:
            print(f"⚠ Robust FID calculation failed: {e}")
            return float('inf')
    
    def _calculate_statistical_distance(self, real_images, fake_images):
        """Métrique de fallback basée sur les statistiques d'images"""
        try:
            # Conversion en numpy
            real_np = real_images.detach().cpu().numpy()
            fake_np = fake_images.detach().cpu().numpy()
            
            # Statistiques de base
            real_mean = np.mean(real_np)
            fake_mean = np.mean(fake_np)
            real_std = np.std(real_np)
            fake_std = np.std(fake_np)
            
            # Distance statistique simple
            mean_diff = abs(real_mean - fake_mean)
            std_diff = abs(real_std - fake_std)
            
            # Score combiné (plus bas = meilleur)
            stat_distance = mean_diff * 100 + std_diff * 50
            
            return float(stat_distance)
            
        except Exception as e:
            print(f"⚠ Statistical distance calculation failed: {e}")
            return 100.0  # Score par défaut
    
    def calculate_diversity_score(self, fake_images):
        """Score de diversité des images générées"""
        try:
            if fake_images.numel() == 0:
                return 0.0
            
            # Calcul de la diversité basée sur la variance des pixels
            fake_flat = fake_images.view(fake_images.size(0), -1)
            
            # Variance moyenne entre images
            pairwise_distances = torch.cdist(fake_flat, fake_flat)
            
            # Exclusion de la diagonale (distance à soi-même)
            mask = ~torch.eye(pairwise_distances.size(0), dtype=torch.bool)
            avg_distance = pairwise_distances[mask].mean()
            
            return float(avg_distance)
            
        except Exception as e:
            print(f"⚠ Diversity score calculation failed: {e}")
            return 0.0
    
    def calculate_quality_score(self, images):
        """Score de qualité basé sur la netteté et le contraste"""
        try:
            if images.numel() == 0:
                return 0.0
            
            # Conversion en format approprié
            imgs = torch.clamp((images + 1) / 2, 0, 1)
            
            # Calcul du gradient (netteté)
            grad_x = torch.abs(imgs[:, :, :, 1:] - imgs[:, :, :, :-1])
            grad_y = torch.abs(imgs[:, :, 1:, :] - imgs[:, :, :-1, :])
            
            sharpness = (grad_x.mean() + grad_y.mean()) / 2
            
            # Calcul du contraste
            contrast = torch.std(imgs)
            
            # Score combiné
            quality = (sharpness * 10 + contrast * 5).item()
            
            return quality
            
        except Exception as e:
            print(f"⚠ Quality score calculation failed: {e}")
            return 0.0

# Configuration mise à jour
class ImprovedModelConfig:
    def __init__(self):
        self.experiment_name = f"cifar10_robust_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.model_registry_path = Path("model_registry")
        self.metrics_path = Path("metrics")
        self.artifacts_path = Path("artifacts")
        
        for path in [self.model_registry_path, self.metrics_path, self.artifacts_path]:
            path.mkdir(exist_ok=True)
        
        self.early_stopping_patience = 8  # Plus conservateur
        self.model_selection_metric = 'composite_score'
        self.evaluation_frequency = 3  # Évaluation tous les 3 epochs
        self.metrics_to_track = ['fid_score', 'diversity_score', 'quality_score', 'composite_score']

# Classe de monitoring améliorée
class ImprovedModelMonitor:
    """Monitoring amélioré avec métriques robustes"""
    
    def __init__(self, config):
        self.config = config
        self.metrics_history = defaultdict(list)
        self.resource_history = defaultdict(deque)
        self.best_models = {}
        self.early_stopping_counter = 0
        self.best_score = float('inf')
        self.metrics_calculator = RobustGANMetrics(device)
        
        # Métriques alternatives pour Kaggle
        self.alternative_metrics = [
            'statistical_distance', 'diversity_score', 'quality_score', 
            'loss_stability', 'gradient_norm'
        ]
    
    def comprehensive_evaluation(self, generator, real_samples, epoch):
        """Évaluation complète avec métriques multiples"""
        try:
            generator.eval()
            metrics = {}
            
            with torch.no_grad():
                # Génération d'échantillons
                eval_noise = torch.randn(min(100, len(real_samples)), latent_dim, 1, 1, device=device)
                fake_samples = generator(eval_noise)
                
                # Limitation du nombre d'échantillons pour éviter les problèmes mémoire
                n_samples = min(50, len(real_samples), len(fake_samples))
                real_subset = real_samples[:n_samples]
                fake_subset = fake_samples[:n_samples]
                
                # Métriques principales
                start_time = time.time()
                
                # FID robuste
                fid_score = self.metrics_calculator.calculate_lightweight_fid(
                    real_subset, fake_subset
                )
                metrics['fid_score'] = fid_score
                
                # Métriques alternatives
                diversity = self.metrics_calculator.calculate_diversity_score(fake_subset)
                quality = self.metrics_calculator.calculate_quality_score(fake_subset)
                
                metrics.update({
                    'diversity_score': diversity,
                    'quality_score': quality,
                    'inference_time': time.time() - start_time,
                    'n_samples_evaluated': n_samples
                })
                
                # Score composite pour sélection de modèle
                if not np.isnan(fid_score) and not np.isinf(fid_score):
                    composite_score = fid_score
                else:
                    # Fallback: utiliser le score de qualité inversé
                    composite_score = max(0, 100 - quality * 10)
                
                metrics['composite_score'] = composite_score
                
                print(f"📊 Epoch {epoch} Metrics:")
                print(f"   FID: {fid_score:.3f}")
                print(f"   Diversity: {diversity:.3f}")
                print(f"   Quality: {quality:.3f}")
                print(f"   Composite: {composite_score:.3f}")
                
                return metrics
                
        except Exception as e:
            print(f"⚠ Comprehensive evaluation failed: {e}")
            # Métriques par défaut en cas d'erreur
            return {
                'fid_score': 100.0,
                'diversity_score': 0.0,
                'quality_score': 0.0,
                'composite_score': 100.0,
                'inference_time': 0.0,
                'evaluation_error': str(e)
            }
    
    def check_early_stopping(self, current_score):
        """Early stopping basé sur le score composite"""
        if np.isnan(current_score) or np.isinf(current_score):
            current_score = float('inf')
            
        if current_score < self.best_score:
            self.best_score = current_score
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return self.early_stopping_counter >= self.config.early_stopping_patience
    
    def log_metrics(self, epoch, metrics_dict):
        """Logging robuste des métriques"""
        timestamp = datetime.now().isoformat()
        
        # Nettoyage des valeurs NaN/Inf
        clean_metrics = {}
        for key, value in metrics_dict.items():
            if isinstance(value, (int, float)):
                if np.isnan(value) or np.isinf(value):
                    clean_metrics[key] = None
                else:
                    clean_metrics[key] = float(value)
            else:
                clean_metrics[key] = value
        
        log_entry = {
            'timestamp': timestamp,
            'epoch': epoch,
            **clean_metrics
        }
        
        # Sauvegarde des métriques
        for key, value in clean_metrics.items():
            if value is not None:
                self.metrics_history[key].append(value)
        
        # Sauvegarde JSON
        metrics_file = self.config.metrics_path / f"{self.config.experiment_name}_metrics.jsonl"
        with open(metrics_file, 'a') as f:
            f.write(json.dumps(log_entry, default=str) + '\n')
        
        print(f"📊 Epoch {epoch}: {clean_metrics}")
    
    def save_model_checkpoint(self, generator, discriminator, epoch, metrics, is_best=False):
        """Sauvegarde robuste avec validation"""
        try:
            checkpoint_dir = self.config.model_registry_path / f"epoch_{epoch}"
            checkpoint_dir.mkdir(exist_ok=True)
            
            # Validation des modèles avant sauvegarde
            if not hasattr(generator, 'state_dict') or not hasattr(discriminator, 'state_dict'):
                raise ValueError("Invalid model objects for saving")
            
            # Métadonnées avec validation
            clean_metrics = {k: v for k, v in metrics.items() 
                           if not (isinstance(v, float) and (np.isnan(v) or np.isinf(v)))}
            
            metadata = {
                'epoch': epoch,
                'timestamp': datetime.now().isoformat(),
                'metrics': clean_metrics,
                'model_size_mb': self._get_model_size(generator) + self._get_model_size(discriminator),
                'is_best': is_best,
                'config': {
                    'latent_dim': latent_dim,
                    'image_channels': 3,  # For CIFAR-10
                    'learning_rate': lr_finetune
                }
            }
            
            # Sauvegarde des modèles
            torch.save(generator.state_dict(), checkpoint_dir / "generator.pth")
            torch.save(discriminator.state_dict(), checkpoint_dir / "discriminator.pth")
            
            # Sauvegarde des métadonnées
            with open(checkpoint_dir / "metadata.json", 'w') as f:
                json.dump(metadata, f, indent=2, default=str)
            
            # Marquage du meilleur modèle
            if is_best:
                best_model_path = self.config.model_registry_path / "best_model"
                if best_model_path.exists():
                    import shutil
                    shutil.rmtree(best_model_path)
                
                import shutil
                shutil.copytree(checkpoint_dir, best_model_path)
                
                composite_score = clean_metrics.get('composite_score', 'N/A')
                print(f"🏆 New best model saved (Composite Score: {composite_score})")
                
        except Exception as e:
            print(f"⚠ Model checkpoint saving failed: {e}")
    
    def _get_model_size(self, model):
        """Calcul robuste de la taille du modèle"""
        try:
            param_size = sum(p.numel() * p.element_size() for p in model.parameters())
            buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
            return (param_size + buffer_size) / 1024**2
        except Exception as e:
            print(f"⚠ Model size calculation failed: {e}")
            return 0.0
    
    def monitor_resources(self):
        """Surveillance des ressources système"""
        try:
            resources = {
                'cpu_usage': psutil.cpu_percent(),
                'memory_usage': psutil.virtual_memory().percent,
                'timestamp': datetime.now().isoformat()
            }
            
            if torch.cuda.is_available():
                resources.update({
                    'gpu_memory_used': torch.cuda.memory_allocated() / 1024**3,
                    'gpu_memory_total': torch.cuda.get_device_properties(0).total_memory / 1024**3
                })
            
            return resources
            
        except Exception as e:
            print(f"⚠ Resource monitoring failed: {e}")
            return {}

# Tests Kaggle améliorés
def improved_kaggle_test_suite():
    """Suite de tests robuste pour Kaggle"""
    print("\n🧪 Running improved Kaggle tests...")
    
    try:
        # Test 1: Vérification système
        print(f"🖥 Python version: {sys.version.split()[0]}")
        print(f"🔥 PyTorch version: {torch.__version__}")
        print(f"🖼 Torchvision version: {torchvision.__version__}")
        
        # Test 2: Mémoire disponible
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
            print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
        
        # Test 3: Test des métriques
        test_metrics = RobustGANMetrics(device)
        test_real = torch.randn(4, 3, 64, 64).to(device)
        test_fake = torch.randn(4, 3, 64, 64).to(device)
        
        fid_test = test_metrics.calculate_lightweight_fid(test_real, test_fake)
        diversity_test = test_metrics.calculate_diversity_score(test_fake)
        quality_test = test_metrics.calculate_quality_score(test_fake)
        
        print(f"🧮 Metrics test - FID: {fid_test:.3f}, Diversity: {diversity_test:.3f}, Quality: {quality_test:.3f}")
        
        # Validation des métriques
        assert not np.isnan(fid_test), f"FID calculation returned NaN"
        assert not np.isnan(diversity_test), f"Diversity calculation returned NaN"
        assert not np.isnan(quality_test), f"Quality calculation returned NaN"
        
        print("✅ All improved Kaggle tests passed!")
        return True
        
    except Exception as e:
        print(f"❌ Kaggle test failed: {e}")
        return False

# Définition des modèles de base (Generator et Discriminator)
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # Input is Z, going into a convolution
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            # state size. 512 x 4 x 4
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            # state size. 256 x 8 x 8
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # state size. 128 x 16 x 16
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # state size. 64 x 32 x 32
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. img_channels x 64 x 64
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # input is (img_channels) x 64 x 64
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 64 x 32 x 32
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 128 x 16 x 16
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 256 x 8 x 8
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 512 x 4 x 4
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialisation des composants
print("\n--- DÉBUT DE LA PHASE 2 INDUSTRIELLE CORRIGÉE ---")

# Initialisation des composants améliorés
config = ImprovedModelConfig()
monitor = ImprovedModelMonitor(config)
metrics_calculator = RobustGANMetrics(device)

print(f"🎯 Experiment: {config.experiment_name}")
print(f"🔧 Using robust metrics with fallback strategies")

# Test initial
test_passed = improved_kaggle_test_suite()
if not test_passed:
    print("⚠ Some tests failed, but continuing with robust fallbacks...")

# --- 2.1. Préparation des données CIFAR-10 ---
image_channels_cifar = 3
transform_cifar = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
dataloader_cifar = DataLoader(cifar_dataset, batch_size=batch_size, shuffle=True)

# Dataset de validation pour métriques
cifar_val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)
val_dataloader = DataLoader(cifar_val_dataset, batch_size=64, shuffle=False)

# Échantillon de vraies images pour calcul FID
real_samples_for_metrics = next(iter(val_dataloader))[0][:100].to(device)

# Initialisation des modèles
generator_cifar = Generator(latent_dim, image_channels_cifar).to(device)
discriminator_cifar = Discriminator(image_channels_cifar).to(device)

# Initialisation des optimiseurs
optimizer_G_cifar = optim.Adam(generator_cifar.parameters(), lr=lr_finetune, betas=betas)
optimizer_D_cifar = optim.Adam(discriminator_cifar.parameters(), lr=lr_finetune, betas=betas)

# Fonction de perte
criterion = nn.BCELoss()

# --- 2.3. Fine-Tuning avec monitoring industriel ---
print(f"\n🚀 Starting industrial fine-tuning...")
print(f"📊 Tracking metrics: {config.metrics_to_track}")

# Variables de tracking
training_start_time = time.time()
epoch_times = []

for epoch in range(epochs_finetune):
    epoch_start_time = time.time()
    
    # Métriques d'époque
    epoch_metrics = {
        'loss_d': 0.0,
        'loss_g': 0.0,
        'loss_d_real': 0.0,
        'loss_d_fake': 0.0
    }
    
    generator_cifar.train()
    discriminator_cifar.train()
    
    for i, (real_images, _) in enumerate(dataloader_cifar):
        real_images = real_images.to(device)
        batch_size_current = real_images.size(0)
        
        real_labels = torch.ones(batch_size_current, device=device)
        fake_labels = torch.zeros(batch_size_current, device=device)

        # --- Entraînement Discriminateur ---
        optimizer_D_cifar.zero_grad()
        
        d_output_real = discriminator_cifar(real_images)
        # Reshape output to match target shape
        d_output_real = d_output_real.view(-1)  # [batch_size, 1, 1, 1] -> [batch_size]
        errD_real = criterion(d_output_real, real_labels)
        errD_real.backward()
        
        noise = torch.randn(batch_size_current, latent_dim, 1, 1, device=device)
        fake_images = generator_cifar(noise)
        d_output_fake = discriminator_cifar(fake_images.detach())
        # Reshape output to match target shape
        d_output_fake = d_output_fake.view(-1)  # [batch_size, 1, 1, 1] -> [batch_size]
        errD_fake = criterion(d_output_fake, fake_labels)
        errD_fake.backward()
        
        errD = errD_real + errD_fake
        optimizer_D_cifar.step()

        # --- Entraînement Générateur ---
        optimizer_G_cifar.zero_grad()
        d_output_on_fake = discriminator_cifar(fake_images)
        # Reshape output to match target shape
        d_output_on_fake = d_output_on_fake.view(-1)  # [batch_size, 1, 1, 1] -> [batch_size]
        errG = criterion(d_output_on_fake, real_labels)
        errG.backward()
        optimizer_G_cifar.step()
        
        # Accumulation des métriques
        epoch_metrics['loss_d'] += errD.item()
        epoch_metrics['loss_g'] += errG.item()
        epoch_metrics['loss_d_real'] += errD_real.item()
        epoch_metrics['loss_d_fake'] += errD_fake.item()
        
        # Monitoring des ressources (toutes les 50 itérations)
        if i % 50 == 0:
            resources = monitor.monitor_resources()
            
        # Nettoyage mémoire périodique
        if i % 100 == 0:
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            gc.collect()
    
    # Calcul des moyennes d'époque
    num_batches = len(dataloader_cifar)
    for key in epoch_metrics:
        epoch_metrics[key] /= num_batches
    
    epoch_time = time.time() - epoch_start_time
    epoch_times.append(epoch_time)
    
    # Évaluation périodique
    if epoch % config.evaluation_frequency == 0 or epoch == epochs_finetune - 1:
        # Évaluation complète avec métriques robustes
        evaluation_metrics = monitor.comprehensive_evaluation(
            generator_cifar, real_samples_for_metrics, epoch
        )
        
        # Métriques d'entraînement
        complete_metrics = {
            **epoch_metrics,
            **evaluation_metrics,
            'epoch_time': epoch_time,
            'lr_g': optimizer_G_cifar.param_groups[0]['lr'],
            'lr_d': optimizer_D_cifar.param_groups[0]['lr']
        }
        
        # Logging et sauvegarde
        monitor.log_metrics(epoch, complete_metrics)
        
        # Sélection basée sur le score composite
        composite_score = evaluation_metrics.get('composite_score', float('inf'))
        is_best = composite_score < monitor.best_score
        
        monitor.save_model_checkpoint(
            generator_cifar, discriminator_cifar, 
            epoch, complete_metrics, is_best
        )
        
        # Early stopping
        if monitor.check_early_stopping(composite_score):
            print(f"🛑 Early stopping at epoch {epoch}")
            break
    
    # Affichage des métriques
    print(f"[Epoch {epoch+1}/{epochs_finetune}] "
          f"Loss_D: {epoch_metrics['loss_d']:.4f} "
          f"Loss_G: {epoch_metrics['loss_g']:.4f} "
          f"Time: {epoch_time:.1f}s")

# --- Analyse finale ---
training_time = time.time() - training_start_time

print(f"\n🎯 Training completed in {training_time/3600:.2f} hours")
print(f"⚡ Average epoch time: {np.mean(epoch_times):.1f}s")

# Génération du rapport final
final_report = {
    'experiment_name': config.experiment_name,
    'total_training_time_hours': training_time / 3600,
    'total_epochs_trained': epoch + 1,
    'best_composite_score': monitor.best_score,
    'avg_epoch_time_seconds': np.mean(epoch_times),
    'final_metrics': {k: v[-1] for k, v in monitor.metrics_history.items()},
    'early_stopped': monitor.early_stopping_counter >= config.early_stopping_patience
}

# Sauvegarde du rapport
report_path = config.artifacts_path / "final_report.json"
with open(report_path, 'w') as f:
    json.dump(final_report, f, indent=2, default=str)

print(f"📋 Final report saved to: {report_path}")
print(f"🏆 Best model composite score: {monitor.best_score:.3f}")

print("\n--- FIN DE LA PHASE 2 INDUSTRIELLE ---")

In [7]:
import torch
import torch.nn as nn
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
import json
from pathlib import Path

# Définition des classes (identiques à votre code original)
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Fonction pour charger le meilleur modèle
def load_best_model(model_registry_path="model_registry"):
    """Charge le meilleur modèle sauvegardé"""
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Chemin vers le meilleur modèle
    #best_model_path = Path(model_registry_path) / "best_model"
    best_model_path = Path("/kaggle/working/model_registry/best_model")
    if not best_model_path.exists():
        raise FileNotFoundError(f"Meilleur modèle non trouvé dans {best_model_path}")
    
    # Chargement des métadonnées
    with open(best_model_path / "metadata.json", 'r') as f:
        metadata = json.load(f)
    
    print(f"📋 Chargement du meilleur modèle:")
    print(f"   Epoch: {metadata['epoch']}")
    print(f"   Composite Score: {metadata['metrics']['composite_score']:.3f}")
    print(f"   FID Score: {metadata['metrics']['fid_score']:.3f}")
    
    # Paramètres du modèle
    latent_dim = metadata['config']['latent_dim']
    img_channels = metadata['config']['image_channels']
    
    # Initialisation des modèles
    generator = Generator(latent_dim, img_channels).to(device)
    discriminator = Discriminator(img_channels).to(device)
    
    # Chargement des poids
    generator.load_state_dict(torch.load(best_model_path / "generator.pth", map_location=device))
    discriminator.load_state_dict(torch.load(best_model_path / "discriminator.pth", map_location=device))
    
    # Mode évaluation
    generator.eval()
    discriminator.eval()
    
    print("✅ Meilleur modèle chargé avec succès!")
    
    return generator, discriminator, metadata

# Fonction pour générer des images
def generate_images(generator, num_images=16, latent_dim=100, save_path="generated_samples.png"):
    """Génère des images avec le meilleur modèle"""
    
    device = next(generator.parameters()).device
    
    with torch.no_grad():
        # Génération de bruit aléatoire
        noise = torch.randn(num_images, latent_dim, 1, 1, device=device)
        
        # Génération d'images
        fake_images = generator(noise)
        
        # Normalisation pour affichage [0,1]
        fake_images = (fake_images + 1) / 2
        
        # Sauvegarde
        save_image(fake_images, save_path, nrow=4, normalize=True)
        
        print(f"💾 {num_images} images générées et sauvegardées dans {save_path}")
        
        return fake_images

# Exemple d'utilisation complète
def main():
    """Exemple d'utilisation du meilleur modèle"""
    
    try:
        # 1. Chargement du meilleur modèle
        generator, discriminator, metadata = load_best_model()
        
        # 2. Génération d'images
        print("\n🎨 Génération d'images...")
        generated_images = generate_images(
            generator, 
            num_images=16, 
            latent_dim=100,
            save_path="best_model_samples.png"
        )
        
        # 3. Affichage des métriques du modèle
        print(f"\n📊 Métriques du meilleur modèle:")
        for metric, value in metadata['metrics'].items():
            if isinstance(value, float):
                print(f"   {metric}: {value:.3f}")
        
        # 4. Génération d'une grille personnalisée
        print("\n🖼 Création d'une grille personnalisée...")
        with torch.no_grad():
            device = next(generator.parameters()).device
            noise = torch.randn(25, 100, 1, 1, device=device)  # 5x5 grille
            samples = generator(noise)
            samples = (samples + 1) / 2  # Normalisation
            
            # Sauvegarde de la grille
            grid = make_grid(samples, nrow=5, normalize=True, padding=2)
            save_image(grid, "custom_grid_5x5.png")
            print("💾 Grille 5x5 sauvegardée dans custom_grid_5x5.png")
        
        # 5. Évaluation d'images réelles vs générées
        print("\n🔍 Test de discrimination...")
        with torch.no_grad():
            # Images générées
            fake_batch = generator(torch.randn(4, 100, 1, 1, device=device))
            fake_scores = discriminator(fake_batch)
            
            print(f"   Scores de discrimination (fake): {fake_scores.mean().item():.3f}")
            print(f"   (Plus proche de 0.5 = meilleur équilibre G/D)")
        
        return generator, discriminator
        
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        return None, None

# Fonction pour comparer différents checkpoints
def compare_checkpoints(model_registry_path="model_registry"):
    """Compare les métriques de différents checkpoints"""
    
    registry_path = Path(model_registry_path)
    
    if not registry_path.exists():
        print("❌ Dossier model_registry non trouvé")
        return
    
    print("📊 Comparaison des checkpoints:")
    print("-" * 60)
    
    checkpoints = []
    
    # Parcours des dossiers d'epochs
    for epoch_dir in sorted(registry_path.glob("epoch_*")):
        metadata_file = epoch_dir / "metadata.json"
        
        if metadata_file.exists():
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
            
            epoch = metadata['epoch']
            composite_score = metadata['metrics'].get('composite_score', 'N/A')
            fid_score = metadata['metrics'].get('fid_score', 'N/A')
            
            checkpoints.append({
                'epoch': epoch,
                'composite_score': composite_score,
                'fid_score': fid_score,
                'path': epoch_dir
            })
    
    # Affichage trié par composite score
    checkpoints.sort(key=lambda x: x['composite_score'] if isinstance(x['composite_score'], float) else float('inf'))
    
    for i, cp in enumerate(checkpoints[:10]):  # Top 10
        marker = "🏆" if i == 0 else f"{i+1:2d}."
        print(f"{marker} Epoch {cp['epoch']:2d} | Composite: {cp['composite_score']:6.3f} | FID: {cp['fid_score']:6.3f}")

if __name__ == "__main__":
    # Chargement et utilisation du meilleur modèle
    generator, discriminator = main()
    
    # Comparaison des checkpoints
    print("\n" + "="*60)
    compare_checkpoints()

📋 Chargement du meilleur modèle:
   Epoch: 45
   Composite Score: 71.944
   FID Score: 71.944
✅ Meilleur modèle chargé avec succès!

🎨 Génération d'images...
💾 16 images générées et sauvegardées dans best_model_samples.png

📊 Métriques du meilleur modèle:
   loss_d: 0.253
   loss_g: 4.853
   loss_d_real: 0.122
   loss_d_fake: 0.131
   fid_score: 71.944
   diversity_score: 54.729
   quality_score: 1.133
   inference_time: 0.429
   composite_score: 71.944
   epoch_time: 47.342
   lr_g: 0.000
   lr_d: 0.000

🖼 Création d'une grille personnalisée...
💾 Grille 5x5 sauvegardée dans custom_grid_5x5.png

🔍 Test de discrimination...
   Scores de discrimination (fake): 0.026
   (Plus proche de 0.5 = meilleur équilibre G/D)

📊 Comparaison des checkpoints:
------------------------------------------------------------
🏆 Epoch 45 | Composite: 71.944 | FID: 71.944
 2. Epoch 39 | Composite: 73.558 | FID: 73.558
 3. Epoch 54 | Composite: 74.668 | FID: 74.668
 4. Epoch 66 | Composite: 75.651 | FID: 75.651